#**Listar arquivos da pasta Carteiras  (tem validação de número de arquivos em Carteiras)**

In [ ]:
import os
import sys

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
diretorio='/content/drive/MyDrive/Carteiras/'

In [ ]:
os.listdir(diretorio)

['SALDOS_C.TXT',
 'ESC_565.CSV',
 'ESC_ESSEN_565.CSV',
 'TB_BATIMENTO_HISOFI4.xlsx',
 'TB_BATIMENTO_HISOFI3.xlsx',
 'TB_BATIMENTO_HISOFI2.xlsx',
 'TB_BATIMENTO_HISOFI1.xlsx',
 'TB_BATIMENTO_HISOFI.xlsx',
 'TB_DIST_HISOFI.xlsx',
 'TABELA_ESSENCIAL_HISOFI.xlsx',
 'ASIG CARTERA OCT - HISOFI - 13.10.2023.xlsx',
 'Asignacion20231016.xlsx',
 'Rep_CarteraApha_16102023_1220.csv',
 '1691077668699template_sofi.xlsx']

In [ ]:
len(os.listdir(diretorio))

14

In [ ]:
if len(os.listdir(diretorio))!=14:
  sys.exit()

# **Triagem dos Arquivos baixados**


## Inicialização

In [ ]:
import pandas as pd
import numpy as np
import os
import datetime
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Apagar COLLECTI.ZIP se existir

In [ ]:
mydrive='/content/drive/MyDrive/'
dir_carteiras=mydrive+'Carteiras/'
#Remover COLLECTI.ZIP
try:
  os.remove(mydrive+'COLLECTI.ZIP')
except:
  print('Arquivo COLLECTI.ZIP não existente')
#Remover SALDOS_C.ZIP
try:
  os.remove(mydrive+'SALDOS_C.ZIP')
except:
  print('Arquivo SALDOS_C.ZIP não existente')

Arquivo COLLECTI.ZIP não existente
Arquivo SALDOS_C.ZIP não existente


##Importando Tabela de Diretórios

In [ ]:
xlsx_diretorios=pd.read_excel(mydrive+'Processamentos/Tratamento/auxiliary/Diretorios.xlsx',sheet_name='Arquivos',dtype=object)
nomes=np.array(xlsx_diretorios['Nome']).astype(str)
carteiras=np.array(xlsx_diretorios['Carteira'])
diretorios=np.array(xlsx_diretorios['Diretório'])
print(nomes)

['Asignacion' '1800' '1801' 'TABELA_ESSENCIAL_HISOFI' 'ESC_ESSEN_565'
 'ASIG CARTERA' 'Rep_CarteraApha' 'TB_DIST' 'ESC_565' 'TB_BATIMENTO'
 'COLLECTI' 'Plantón' 'Descuentos' 'natura-mex-padrao'
 'natura-peru-exclusiva' 'ESPECIAL' 'natura-mex-tardia'
 'pronto-carteira-1' 'porto-essencial' 'porto-tradicional' 'SALDOS_C'
 'Late Stage' 'Descuentos' 'template_sofi']


##Movendo arquivos para as pastas corretas

In [ ]:
rodar=[]
for arquivo in os.listdir(dir_carteiras):
  print(arquivo)
  for i in range(len(nomes)):
    if arquivo.find(nomes[i])>=0:
      rodar.append(carteiras[i])
      os.makedirs(diretorios[i],exist_ok=True)
      os.rename(dir_carteiras+arquivo,diretorios[i]+arquivo)
      print(arquivo,'movido para',diretorios[i])
      break

SALDOS_C.TXT
ESC_565.CSV
ESC_ESSEN_565.CSV
TB_BATIMENTO_HISOFI4.xlsx
TB_BATIMENTO_HISOFI3.xlsx
TB_BATIMENTO_HISOFI2.xlsx
TB_BATIMENTO_HISOFI1.xlsx
TB_BATIMENTO_HISOFI.xlsx
TB_DIST_HISOFI.xlsx
TABELA_ESSENCIAL_HISOFI.xlsx
ASIG CARTERA OCT - HISOFI - 13.10.2023.xlsx
Asignacion20231016.xlsx
Rep_CarteraApha_16102023_1220.csv
1691077668699template_sofi.xlsx


##Carteiras que tiveram arquivos movidos

In [ ]:
rodar=np.array(rodar)
rodar=np.unique(rodar)
rodar=np.sort(rodar)
print(list(rodar),len(rodar))

['Avon PE', 'Natura MEX', 'Natura PE', 'Porto Batimento', 'Porto Essencial', 'Pronto', 'Risel'] 7


# **Tratamento NATURA PE EXCLUSIVA**

## Inicialização

In [ ]:
tem_especial=True
mover=True

In [ ]:
import pandas as pd
import numpy as np
import os
import datetime
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
carteira='Natura PE'

#Definindo diretórios
diretorio_tratamento=r'/content/drive/MyDrive/Processamentos/Tratamento/'
diretorio=diretorio_tratamento + carteira + '/'
output='/'.join(['/content/drive/MyDrive/Pronto pra upload',str(datetime.date.today().year),str(datetime.date.today().month).zfill(2),datetime.date.today().strftime('%d-%m-%Y')])+'/'
os.makedirs(output,exist_ok=True)
ingestion=diretorio+'ingestion/'
planton=diretorio+'Plantón/'
processedfiles=diretorio+'processedfiles/'
dir_especial=diretorio+'Especial/'
dir_num_errados=diretorio+'Números Inválidos/'
os.makedirs(dir_num_errados,exist_ok=True)

today=datetime.date.today()
#Nome do arquivo de saída com caminho
csv_saida=output+'early_natura-peru-exclusiva_'+str(today.strftime('%d-%m-%Y'))+'.csv'
#Arquivo com números inválidos
errados_saida=dir_num_errados+'numeros-errados_natura-pe-exclusiva_'+str(today)+'.xlsx'

In [ ]:
#Esperando ler 1
num_arquivo=1

while len(os.listdir(ingestion))!=num_arquivo:
  time.sleep(5)
print(os.listdir(ingestion))

['ASIG CARTERA OCT - HISOFI - 13.10.2023.xlsx']


In [ ]:
#Identificando as planilhas EXCLUSIVA e ESPECIAL
for arquivo in os.listdir(ingestion):
    if arquivo.find('ASIG')>=0:
      nome_exclusiva=arquivo
      arquivo_exclusiva=ingestion+arquivo

#Identificando o arquivo mais recente da pasta Exclusiva
if tem_especial:
  df_arquivo_recente=pd.DataFrame({'Nome':os.listdir(dir_especial)})
  tempo_modif=np.empty(len(df_arquivo_recente))
  i=0
  for arquivo in os.listdir(dir_especial):
    tempo_modif[i]=os.path.getmtime(dir_especial+arquivo)
    i+=1
  df_arquivo_recente['Tempo de Modificacao']=tempo_modif
  df_arquivo_recente=df_arquivo_recente.sort_values(by='Tempo de Modificacao',ascending=False)
  df_arquivo_recente.index=range(len(df_arquivo_recente))
  print(df_arquivo_recente)
  nome_arquivo_especial=str(df_arquivo_recente['Nome'][0])
  arquivo_especial=dir_especial+nome_arquivo_especial
  print(arquivo_especial)

#Identificando o arquivo mais recente da pasta Plantón
df_arquivo_recente=pd.DataFrame({'Nome':os.listdir(planton)})
tempo_modif=np.empty(len(df_arquivo_recente))
i=0
for arquivo in os.listdir(planton):
  tempo_modif[i]=os.path.getmtime(planton+arquivo)
  i+=1
df_arquivo_recente['Tempo de Modificacao']=tempo_modif
df_arquivo_recente=df_arquivo_recente.sort_values(by='Tempo de Modificacao',ascending=False)
df_arquivo_recente.index=range(len(df_arquivo_recente))
print(df_arquivo_recente)
nome_arquivo_platon=str(df_arquivo_recente['Nome'][0])
arquivo_planton=planton+nome_arquivo_platon
print(arquivo_planton)

#csv_saida2=arquivo_exclusiva.replace('.xlsx',' - Tratado com Planilha.csv').replace(ingestion,output)
#xlsx_saida=arquivo_exclusiva.replace('.xlsx',' - Tratado.xlsx').replace(ingestion,output)

                                                Nome  Tempo de Modificacao
0  ASIG CARTERA JUL- HISOFI - ESPECIAL - 14.07.20...          1.690397e+09
/content/drive/MyDrive/Processamentos/Tratamento/Natura PE/Especial/ASIG CARTERA JUL- HISOFI - ESPECIAL - 14.07.2023.xlsx
                                                Nome  Tempo de Modificacao
0  CNs Plantón C12- 2023 - Actualizado 23 Agosto...          1.692904e+09
/content/drive/MyDrive/Processamentos/Tratamento/Natura PE/Plantón/CNs Plantón C12- 2023 - Actualizado 23 Agosto - HISOFI.xlsx


In [ ]:
if tem_especial:
    print(arquivo_exclusiva,'\n',arquivo_especial,'\n',arquivo_planton)
else:
    print(arquivo_exclusiva,'\n',arquivo_planton)

/content/drive/MyDrive/Processamentos/Tratamento/Natura PE/ingestion/ASIG CARTERA OCT - HISOFI - 13.10.2023.xlsx 
 /content/drive/MyDrive/Processamentos/Tratamento/Natura PE/Especial/ASIG CARTERA JUL- HISOFI - ESPECIAL - 14.07.2023.xlsx 
 /content/drive/MyDrive/Processamentos/Tratamento/Natura PE/Plantón/CNs Plantón C12- 2023 - Actualizado 23 Agosto - HISOFI.xlsx


In [ ]:
#Carregando Planilhas
exclusiva=pd.read_excel(arquivo_exclusiva,dtype=object)
exclusiva['Planilha']='EXCLUSIVA'
print('Exclusiva carregado com sucesso!')
df_planton=pd.read_excel(arquivo_planton,dtype=object)
print('Plantón carregado com sucesso!')
if tem_especial:
    especial=pd.read_excel(arquivo_especial,dtype=object)
    print('Especial carregado com sucesso!')
    especial['Planilha']='ESPECIAL'
else:
    print('Carteira Especial não existente')

Exclusiva carregado com sucesso!
Plantón carregado com sucesso!
Especial carregado com sucesso!


## Início das Transformações

### Concatenar Exclusiva e Especial

In [ ]:
#Concatenando Carteiras e criando cópias
if tem_especial:
    concatenar=pd.concat([exclusiva,especial],join='outer',ignore_index=True)
    cartera=concatenar.copy()
else:
    cartera=exclusiva.copy()
planton=df_planton.copy()

###Filtrar descontos da Planton por data

In [ ]:
planton['Fecha Vigencia'] = pd.to_datetime(planton['Fecha Vigencia'])
filtro=planton['Fecha Vigencia']>=pd.Timestamp(today)
planton=planton[filtro]

### Capitalizar 'Nombre de la Persona'

In [ ]:
#Capitalizando coluna de Nomes
cartera['Nombre de la Persona']=cartera['Nombre de la Persona'].str.title()
cartera['Nombre de la Persona']

0                            Irma  Lomas Garcia
1                  Lina  Guadalupe Flores Ramos
2                Jecika Elizabeth  Hidalgo Celi
3                          Hector  Quenaya Illa
4                Sandra Lucia  Vidaurre Cajusol
                          ...                  
45876                  Diana Nicol  Pecho Rojas
45877             Luis Antonio  Arcentales Ruiz
45878          Josselin Xiomara  Falcon Riveros
45879           Kateryn Veronica  Cruz Guerrero
45880    Tatihana Loren Ruth  Sernaque Parrilla
Name: Nombre de la Persona, Length: 45881, dtype: object

### Tratando '% Descuento Saldo Original' para ficar que nem no excel em %

In [ ]:
#Colocando % na coluna de Descontos e multiplicando por 100
cartera['% Descuento Saldo Original']=cartera['% Descuento Saldo Original'].astype(float)*100
cartera['% Descuento Saldo Original']=cartera['% Descuento Saldo Original'].astype(int)
cartera['% Descuento Saldo Original']=cartera['% Descuento Saldo Original'].apply(lambda x: str(x)+'%')
cartera['% Descuento Saldo Original']

0        0%
1        0%
2        0%
3        0%
4        0%
         ..
45876    0%
45877    0%
45878    0%
45879    0%
45880    0%
Name: % Descuento Saldo Original, Length: 45881, dtype: object

### Tratando datas da coluna 'Fecha nasc'

In [ ]:
#Tratando a coluna de Fecha Nasc que tem datas em formato de número
coluna='Fecha nasc'
# cartera[coluna]=cartera[coluna].apply(lambda x:'' if str(x)=='-' else x)
data=np.array(cartera[coluna])
for i in range(len(data)):
  if str(data[i]).find('/')==-1 and str(data[i]).find('-')==-1:
      data[i]=str(pd.to_datetime('1900-01-01')+pd.Timedelta(days=data[i]-2))
cartera[coluna]=data
cartera[coluna]=cartera[coluna].apply(lambda x: str(x)[:10])

#Tratando as datas que estão com - para /
x=np.array(cartera[coluna])
print(x)
for i in range(len(x)):
  if x[i].find('/')==-1:
    x[i]=str(x[i])[-2:]+'/'+str(x[i])[5:7]+'/'+str(x[i])[:4]
cartera[coluna]=x
#Datas que são - viram -//- depois das operações acima, substituir essas datas por vazio
cartera[coluna]=cartera[coluna].str.replace('-//-','')
cartera[coluna]

['1973-12-30' '1995-12-30' '1979-05-06' ... '2023-03-28' '2023-03-29'
 '2023-03-29']


0        30/12/1973
1        30/12/1995
2        06/05/1979
3        28/09/1981
4        17/07/1986
            ...    
45876    27/03/2023
45877    28/03/2023
45878    28/03/2023
45879    29/03/2023
45880    29/03/2023
Name: Fecha nasc, Length: 45881, dtype: object

### Tirando o horário das colunas de data e convertendo pro formato brasileiro

In [ ]:
#Tirando horário nas colunas de data
print(cartera.select_dtypes(include='datetime').columns.tolist())
for coluna in cartera.select_dtypes(include='datetime').columns.tolist():
    cartera[coluna]=cartera[coluna].apply(lambda x: str(x)[:10])
    cartera[coluna]=pd.to_datetime(cartera[coluna])
    cartera[coluna]=cartera[coluna].dt.strftime('%d/%m/%Y')

['Fecha Pedido', 'Fecha Vcto', 'Fecha Saldo', 'Fecha Saldo Corregido']


### Remover colunas 'Estructura Comercial','Estructura Comercial Padre'

In [ ]:
#Apagando colunas de Estrutuctura Comercial
cartera=cartera.drop(['Estructura Comercial','Estructura Comercial Padre'],axis=1)

### Remover DNI nulos, zeros e - e preencher com 8 dígitos

In [ ]:
#Tratando DNI nulo e hífens
print('Total de linhas antes:',len(cartera))
nulos=np.array(cartera['DNI'].notnull())
hifens=np.array(cartera['DNI']!='-')
filtro=nulos*hifens
print('Total de linhas diferente de - e nulo:',filtro.sum())
cartera=cartera[filtro]
#Formatando DNI com 8 dígitos
cartera['DNI']=cartera['DNI'].astype(object)
cartera['DNI']=cartera['DNI'].apply(lambda x: str(x).zfill(8))
#Removendo zeros
cartera=cartera[cartera['DNI']!='00000000']
print('Total de linhas depois de filtrar zeros:',len(cartera))
cartera.sort_values('DNI').head(3)

Total de linhas antes: 45881
Total de linhas diferente de - e nulo: 45522
Total de linhas depois de filtrar zeros: 45495


,Orden del Pedidos,Código Persona,Cod Grupo,Nombre de la Persona,Título,No Pedido,No Factura,Fecha Pedido,Fecha Vcto,Fecha Saldo,...,DNI,Dirección,Teléfono Persona,Numero Celular,Correo Electrónico,TRAMO,AGENCIA,RIESGO,COLOR,Planilha
1563,4,2776451,3774,Leiser Abraham Asayag Lozano,15173584,39199919,5257578,16/08/2023,14/09/2023,22/09/2023,...,00000285,MZ 201 LT 13 REF PASANDO AAHH NVOCANAAN PASAJE...,985474066,985474066,leiser.asayag29@gmail.com,A) 1 - 30,HISOFI,BAJO,ORO,EXCLUSIVA
27396,5,2516168,951,Marianella Oxana Gomez Sanchez,11044272,34517034,526074,02/10/2020,02/11/2020,03/10/2020,...,00000445,CALLE TAHUANTINSUYO 0 CÓDIGO POSTAL 0 A UNA CU...,-,941946946,tauto_99@hotmail.com,I) 721 - 1080,HISOFI,-,BRONCE,EXCLUSIVA
27135,1,2533128,1287,Norma Vasquez Alvarado,11029804,34497170,509619,26/09/2020,27/10/2020,29/09/2020,...,00004183,JR CUSCO 7 T 22 - B 0 CÓDIGO POSTAL 0 CASA D...,-,927906668,edyeluz.m.a@gmail.com,I) 721 - 1080,HISOFI,-,BRONCE,EXCLUSIVA


### Adicionando coluna 'MONEDA'

In [ ]:
#Criando coluna MONEDA
cartera['MONEDA']='PEN'

### Mesclar tabela Plantón

In [ ]:
#Mesclando tabelas
codigo_importe=nome_arquivo_platon[13:16] #12:15 no script local
importe_a_pagar='Importe a pagar - Plantón Cobranza ' + codigo_importe     #'Importe a pagar - Plantón Cobranza ' + codigo_importe
planton=planton[['No Pedido',importe_a_pagar]]
#planton.columns=['No Pedido',importe_a_pagar]   #Renomear coluna de Saldo(Antigo Importe a pagar) porque já tem coluna Saldo na carteira
mesclado=cartera.merge(planton,on='No Pedido',how='left')

### Substituindo Monto Mínimo a Pagar pelo valor do Platón

In [ ]:
#Substituindo Monto Mínimo a Pagar pelo valor do Platón
importe_nao_nulo=mesclado[importe_a_pagar].notnull()
importe_nao_nulo.sum()
valores=np.array(mesclado['Monto mínimo a pagar Campaña'])
valores[importe_nao_nulo]=mesclado[importe_nao_nulo][importe_a_pagar]
mesclado['Monto mínimo a pagar Campaña']=valores

###Criar coluna de Descuentos


In [ ]:
mesclado['Descuentos'] = 0
mesclado.loc[importe_nao_nulo,'Descuentos']=1

### Apagar a coluna auxiliar 'Importe a pagar - Plantón Cobranza C10'

In [ ]:
mesclado=mesclado.drop(importe_a_pagar,axis=1)

### Remover 'No Pedido' duplicados

In [ ]:
#Remover Pedidos duplicados
print(len(mesclado))
mesclado=mesclado.drop_duplicates(subset='No Pedido')
print(len(mesclado))

45495
44953


### Arredondando e formatando colunas de números para 2 casas decimais

In [ ]:
#Formatar números para 2 casas decimais
colunas=['Original', 'Saldo','Saldo Corregido','Monto mínimo a pagar Campaña','Interés/multa Acumulados (100% descuento)','Monto Descuento Saldo Original','Total descuentos']
for coluna in colunas:
    mesclado[coluna]=mesclado[coluna].astype(float)
    mesclado[coluna]=mesclado[coluna].round(2)
    mesclado[coluna]=mesclado[coluna].map('{:.2f}'.format)

###Tratamento dos números de telefone

In [ ]:
#Fazendo coerce com Celular e Telefone
telefone=np.array(mesclado['Teléfono Persona'])
celular=np.array(mesclado['Numero Celular'])
celular_nao_nulo=mesclado['Numero Celular'].notnull()
telefone[celular_nao_nulo]=celular[celular_nao_nulo]
mesclado['Numero Celular']=telefone
#Tirando espaços
mesclado['Numero Celular']=mesclado['Numero Celular'].astype(str).str.strip()
#Tratando números com mais de 9 dígitos e 8 dígitos
mesclado['Numero Celular']=mesclado['Numero Celular'].apply(lambda x: '9'+str(x) if len(str(x))==8 else (str(x)[-9:] if len(str(x))>9 else x))
#Apagar números que não começam com 9
mesclado['Numero Celular']=mesclado['Numero Celular'].apply(lambda x: '' if str(x)[0]!='9' else x)
#Apagar números com menos de 8 dígtios
filtro=np.char.str_len(np.char.array(mesclado['Numero Celular']))<8
#Exportar os números errados
mesclado.loc[:,'Numero Celular'][filtro].to_csv(errados_saida,index=False)
#Substituir por '' os números errados
mesclado.loc[:,'Numero Celular'][filtro]=''

<ipython-input-37-e68046e36635>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mesclado.loc[:,'Numero Celular'][filtro]=''


###Tratamento de Email

In [ ]:
# Aplicando lower e strip na coluna de email
coluna_email = 'Correo Electrónico'
mesclado[coluna_email] = mesclado[coluna_email].str.lower().str.strip()
# Apagando emails não validados
mesclado[coluna_email].loc[mesclado[coluna_email].str.find('@') < 0] = ''
mesclado['num @'] = mesclado[coluna_email].str.len() - mesclado[coluna_email].str.replace('@', '').str.len()
filtro = mesclado['num @'] != 1
mesclado.loc[filtro, coluna_email] = ''
mesclado.drop('num @', axis=1, inplace=True)

<ipython-input-38-8d69745d573b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mesclado[coluna_email].loc[mesclado[coluna_email].str.find('@') < 0] = ''


###Tratar coluna de Nome

In [ ]:
coluna_nomes=['Nombre de la Persona']
for coluna_nome in coluna_nomes:
  mesclado[coluna_nome]=mesclado[coluna_nome].str.strip().str.title()

## Exportando

In [ ]:
#mesclado.to_csv(csv_saida2,index=False)
try:
  mesclado=mesclado.drop('Planilha',axis=1)
except:
  print('Coluna planilha já excluída')
#mesclado.to_excel(xlsx_saida,index=False)
mesclado.to_csv(csv_saida,index=False)

## Movendo Somente as Carteiras Exclusivas

In [ ]:
if mover:
  arquivos=[nome_exclusiva]
  for arquivo in arquivos:
    try:
      os.rename(ingestion + arquivo, processedfiles + arquivo)
    except:
      os.remove(processedfiles + arquivo)
      os.rename(ingestion + arquivo, processedfiles + arquivo)

# **Tratamento NATURA MEX PADRÃO**

In [ ]:
mover=True

import pandas as pd
import numpy as np
import os
import time
import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Definindo Diretórios

In [ ]:
carteira='Natura MEX'

#Definindo diretórios
diretorio_tratamento=r'/content/drive/MyDrive/Processamentos/Tratamento/'
diretorio=diretorio_tratamento + carteira + '/'
output='/'.join(['/content/drive/MyDrive/Pronto pra upload',str(datetime.date.today().year),str(datetime.date.today().month).zfill(2),datetime.date.today().strftime('%d-%m-%Y')])+'/'
os.makedirs(output,exist_ok=True)
ingestion=diretorio+'ingestion/'
processedfiles=diretorio+'processedfiles/'
auxiliary=diretorio+'auxiliary/'
dir_num_errados=diretorio+'Números Inválidos/'
os.makedirs(dir_num_errados,exist_ok=True)
#Nome do arquivo de saída com caminho
today=datetime.date.today()
csv_saida=output+'early_natura-mex-padrao_'+str(today.strftime('%d-%m-%Y'))+'.csv'
#Arquivo com números inválidos
errados_saida=dir_num_errados+'numeros-errados_natura-mex-padrao_'+str(today)+'.xlsx'
#Checando arquivos na ingestion
print(os.listdir(ingestion))

['Rep_CarteraApha_16102023_1220.csv']


In [ ]:
#Mapeando o arquivo Rep
nome_rep_cartera=''
while nome_rep_cartera=='':
  for arquivo in os.listdir(ingestion):
    if arquivo[:3]=='Rep':
      nome_rep_cartera=arquivo
  time.sleep(5)

In [ ]:
concatenado=pd.read_csv(auxiliary+'Personas Concatenado.csv',dtype=object)
concatenado

,CN,'IFE/RFC'
0,4190424,ESMRZY00101115M300
1,4190426,RZRGKR96031409M800
2,4190433,GRMZED93030310M600
3,4190437,MNISSB51070630M000
4,4190438,ANAQCN98121520M600
...,...,...
1925186,5091235,LRSNFL76040620M200
1925187,5091236,NCLPMC84031120M000
1925188,5091237,RBVLES01020322M299
1925189,5091238,CRGRLS04110319H400


In [ ]:
df_rep_cartera=pd.read_csv(ingestion+nome_rep_cartera,dtype=object,encoding='utf_8')

##Cópia da carteira principal

In [ ]:
rep_cartera=df_rep_cartera.copy()

##Início das Transformações

In [ ]:
#Capitalizando coluna de nomes
rep_cartera['Nombre']=rep_cartera['Nombre'].str.title()

In [ ]:
#Mesclando Rep_Cartera
rep_cartera_mesclado=rep_cartera.merge(concatenado, left_on='Codigo Persona',right_on='CN',how='left')
rep_cartera_mesclado=rep_cartera_mesclado.drop('CN',axis=1)

In [ ]:
#Número de vazios nas duas carteiras
#print('Número de vazios em Cartera: '+str(cartera_mesclado[r"'IFE/RFC'"].isnull().sum()))
print('Número de vazios em RepCartera: '+str(rep_cartera_mesclado[r"'IFE/RFC'"].isnull().sum()))

Número de vazios em RepCartera: 1420


In [ ]:
#Exportando os Nulos
#rep_cartera_mesclado[rep_cartera_mesclado[r"'IFE/RFC'"].isnull()].to_excel(output + nome_saida_rep_nulos,index=False)

In [ ]:
rep_cartera_mesclado.columns

Index(['INDICE', 'Codigo Persona', 'Nombre', 'Numero Pedido',
       'Fecha Vencimiento', 'Dias de credito', 'Dias de morosidad',
       'Temporalidad Agencias', 'Valor Titulo', 'Saldo Principal',
       'Saldo actualizado', 'Estructura Padre', 'Estructura',
       'Direccion Completa', 'Telefono movil', 'Telefono Residencial',
       'Correo electronico personal', 'Referencia Bancaria',
       'Agencia Asignada MC', 'Estatus MC', 'Dias en la instancia actual',
       ''IFE/RFC''],
      dtype='object')

In [ ]:
#Formatando colunas numéricas para 2 casas decimais
colunas=['Valor Titulo', 'Saldo Principal','Saldo actualizado']
rep_cartera_mesclado[colunas]=rep_cartera_mesclado[colunas].astype(float)
for coluna in colunas:
  rep_cartera_mesclado[coluna]=rep_cartera_mesclado[coluna].map('{:.2f}'.format)
rep_cartera_mesclado[colunas]

,Valor Titulo,Saldo Principal,Saldo actualizado
0,1563.83,1083.29,1354.11
1,1368.23,1367.99,1846.79
2,2542.92,2542.92,3432.94
3,1629.23,1628.69,2035.86
4,1692.89,297.75,401.96
...,...,...,...
7901,1089.45,1089.10,1470.29
7902,1277.35,1276.60,1595.75
7903,1286.69,1186.39,1482.99
7904,1635.04,834.39,1126.43


In [ ]:
#Apagar IFE nulos
rep_cartera_nao_nulos=rep_cartera_mesclado.dropna(subset=r"'IFE/RFC'")

In [ ]:
#Apagar coluna Dias en la instancia actual
rep_cartera_nao_nulos=rep_cartera_nao_nulos.drop('Dias en la instancia actual',axis=1)

###Tratando número de telefones

In [ ]:
celular=np.array(rep_cartera_nao_nulos['Telefono movil']).astype(str)
for i in range(len(celular)):
  if len(celular[i])>10:
    celular[i]=str(celular[i])[-10:]
  elif len(celular[i])<10 or celular[i]==''.zfill(10):
    celular[i]=''

#Substituindo números pelos valores da lista
rep_cartera_nao_nulos.loc[:,'Telefono movil']=celular

###Tratamento de Email

In [ ]:
  ###Tratamento de Email
  # Aplicando lower e strip na coluna de email
  coluna_email = 'Correo electronico personal'
  rep_cartera_nao_nulos[coluna_email] = rep_cartera_nao_nulos[coluna_email].str.lower().str.strip()
  # Apagando emails não validados
  rep_cartera_nao_nulos[coluna_email].loc[rep_cartera_nao_nulos[coluna_email].str.find('@') < 0] = ''
  rep_cartera_nao_nulos['num @'] = rep_cartera_nao_nulos[coluna_email].str.len() - rep_cartera_nao_nulos[coluna_email].str.replace('@', '').str.len()
  filtro = rep_cartera_nao_nulos['num @'] != 1
  rep_cartera_nao_nulos.loc[filtro, coluna_email] = ''
  rep_cartera_nao_nulos.drop('num @', axis=1, inplace=True)

###Renomear colunas

In [ ]:
#Renomeando as colunas
rep_cartera_nao_nulos.columns=['INDICE','CódigoPersona','Nombre','NúmeroPedido','FechaVencimiento','Dias de credito','Dias de morosidad',\
                               'Temporalidad Agencias','ValorTítulo','Saldo Principal','Saldo actualizado','Estructura Padre','Estructura',\
                               'Direccion Completa','Telefono movil','Telefono Residencial','Correo electronico personal','Referencia Bancaria',\
                               'Agencia Asignada MC','Estatus MC','curp']

###Tratar coluna de Nome

In [ ]:
coluna_nomes=['Nombre']
for coluna_nome in coluna_nomes:
  rep_cartera_nao_nulos[coluna_nome]=rep_cartera_nao_nulos[coluna_nome].str.strip().str.title()

##Exportando

###Exportando arquivos de números errados

In [ ]:
#Criando cópia
num_errados=df_rep_cartera.copy()
#Pegando lista de números
numeros=np.array(num_errados['Telefono movil']).astype(str)
#Criando filtro
length=np.char.str_len(numeros)
filtro1=length<10
filtro2=numeros==''.zfill(10)
filtro=filtro1+filtro2
print(filtro1.sum(),filtro2.sum(),filtro.sum(),len(filtro))
#Aplicando filtro
num_errados=num_errados[filtro]
#Exportando arquivo
num_errados.to_excel(errados_saida,index=False)

251 4 255 7906


###Exportando arquivo tratado

In [ ]:
#Exportando arquivo tratado
rep_cartera_nao_nulos.to_csv(csv_saida,index=False)

##Movendo arquivos

In [ ]:
#Movendo arquivos para processedfiles
if mover:
  arquivos=[nome_rep_cartera]
  for arquivo in arquivos:
    try:
      os.rename(ingestion + arquivo, processedfiles + arquivo)
    except:
      os.remove(processedfiles + arquivo)
      os.rename(ingestion + arquivo, processedfiles + arquivo)

# **Tratamento AVON PE PADRAO (sem Collecti)**

## Inicialização

In [ ]:
#ATUALIZAR A LISTA DE DATAS QUE INICIAM AS CAMPANHAS
mover=True
tem_descuentos=False

In [ ]:
import pandas as pd
import numpy as np
import os
import datetime
import time

###Verificar se tem Descuentos

In [ ]:
if datetime.date.today()<=datetime.date(2023,9,30):
  tem_descuentos=True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
carteira='Avon PE'

today=datetime.date.today()
#Definindo diretórios
diretorio_tratamento=r'/content/drive/MyDrive/Processamentos/Tratamento/'
diretorio=diretorio_tratamento + carteira + '/'
output='/'.join(['/content/drive/MyDrive/Pronto pra upload',str(today.year),str(today.month).zfill(2),today.strftime('%d-%m-%Y')])+'/'
os.makedirs(output,exist_ok=True)
ingestion=diretorio+'ingestion/'
processedfiles=diretorio+'processedfiles/'
dir_descuentos=diretorio+'Descuentos/'
auxiliary=diretorio+'auxiliary/'

#Nome do arquivo de saída com caminho
csv_saida=output+'early_avon-pe-padrao_'+str(today.strftime('%d-%m-%Y'))+'.csv'

##Identificar arquivos da auxiliary

In [ ]:
#Emails e Calendario
for arquivo in os.listdir(auxiliary):
    if arquivo.find('Emails')>=0:
        arquivo_emails=auxiliary+arquivo
        nome_emails=arquivo
    elif arquivo.find('Calendario')>=0:
        arquivo_calendario=auxiliary+arquivo
        nome_calendario=arquivo

##Importar arquivo Descuentos

In [ ]:
if tem_descuentos:
  #Identificando o arquivo mais recente da pasta Descuentos
  df_arquivo_recente=pd.DataFrame({'Nome':os.listdir(dir_descuentos)})
  tempo_modif=np.empty(len(df_arquivo_recente))
  i=0
  for arquivo in os.listdir(dir_descuentos):
    tempo_modif[i]=os.path.getmtime(dir_descuentos+arquivo)
    i+=1
  df_arquivo_recente['Tempo de Modificacao']=tempo_modif
  df_arquivo_recente=df_arquivo_recente.sort_values(by='Tempo de Modificacao',ascending=False)
  df_arquivo_recente.index=range(len(df_arquivo_recente))
  print(df_arquivo_recente)
  nome_arquivo_descuentos=str(df_arquivo_recente['Nome'][0])
  arquivo_descuentos=dir_descuentos+nome_arquivo_descuentos
  print(arquivo_descuentos)

##Importar arquivo de Calendário

In [ ]:
df_calendario=pd.read_excel(arquivo_calendario,dtype=object)
#Definir último dia
if today.weekday()==0:
  dias=3
else:
  dias=1
ultimo=today-datetime.timedelta(days=dias)

##Definir se Tem Collecti

In [ ]:
#Checando se tem collecti
if str(ultimo) in np.array(df_calendario['Data'].astype(str)):
  tem_collecti=True
  zona=np.array(df_calendario.loc[df_calendario['Data']==str(ultimo),'Zona']).astype(int)
else:
  tem_collecti=False
print(tem_collecti)
tem_collecti=False

False


##Ingestion

In [ ]:
#Esperando ler 1 ou 2 arquivos na ingestion (depende do parâmetro 'tem_especial')
if tem_collecti:
  num_arquivo=2
else:
  num_arquivo=1

while len(os.listdir(ingestion))!=num_arquivo:
  time.sleep(5)
print(os.listdir(ingestion))

['SALDOS_C.TXT']


##Identificar arquivos da ingestion

In [ ]:
#Identificando arquivo COLLECTI e SALDOS
for arquivo in os.listdir(ingestion):
    if arquivo.find('COLLECTI')>=0:
        arquivo_txt=ingestion+arquivo
        nome_txt=arquivo
    elif arquivo.find('SALDOS_C')>=0:
        arquivo_saldos=ingestion+arquivo
        nome_saldos=arquivo

##Importando Arquivos

In [ ]:
#Carregando Planilha Collecti
if tem_collecti:
  df_txt=pd.read_csv(arquivo_txt,dtype=object,sep='\t', lineterminator='\r',encoding='unicode_escape',encoding_errors='ignore')
#Carregando Planilha Saldos
df_saldos=pd.read_csv(arquivo_saldos,dtype=object,sep='\t', lineterminator='\r',encoding='unicode_escape',encoding_errors='ignore')
#Carregando planilha de descontos se tiver
if tem_descuentos:
  df_descuentos=pd.read_excel(arquivo_descuentos,dtype=object)
#Carregando planilha de Emails
df_emails=pd.read_csv(arquivo_emails,dtype=object)

##Início das Tranformações

###Juntar tabela de Saldos e Collecti

####Criando cópias

In [ ]:
#Criando cópia do txt para tratar
if tem_collecti:
  txt=df_txt.copy()
saldos=df_saldos.copy()
emails=df_emails.copy()
if tem_descuentos:
  descuentos=df_descuentos.copy()

###Atualizar e Exportar planilha de Emails com o arquivo COLLECTI novo

In [ ]:
#Tratando coluna ZONA
if tem_collecti:
  txt['ZONA']=txt['ZONA'].str.replace('\n','').str.strip()
  emails['ZONA']=emails['ZONA'].str.replace('\n','').str.strip()
  #Validação das ZONAs
  txt_zonas=txt.loc[txt['ZONA']!='','ZONA'].unique().astype(int)  #zona do calendário está em int tbm
  for txt_zona in txt_zonas:
    if txt_zona not in zona:
      print('ZONA do Calendário não está no COLLECTI')
      import sys
      sys.exit()
  for x in zona:
    if not x in txt_zonas:
      print('ZONA do COLLECTI não está no Calendário')
      sys.exit()

In [ ]:
if tem_collecti:
  #Retirar Emails do Collecti novo do arquivo antigo de Emails
  filtro=np.logical_not(np.isin(emails.ZONA.astype(int),zona))  #transformar em int tira os epaços antes
  emails=emails[filtro]
  #Agrupar emails por external ID
  novo=pd.concat([emails,txt])
  print('Antes',len(novo))
  novo=novo.groupby(['ZONA','CARTA','SEC','EQUIPO','TERRITORIO'],as_index=False).last()
  print('Depois',len(novo))

###Selecionar colunas da Tabela de Emails

In [ ]:
emails=emails[['ZONA','CARTA','SEC','EQUIPO','TERRITORIO','E-MAIL']]

####Tratando coluna de Zonas

In [ ]:
#Tratando ZONA
saldos['ZONA']=saldos['ZONA'].str.replace('\n','').str.strip()

####Eliminando ZONAS da tabela SALDOS que estão na COLLECTI

In [ ]:
if tem_collecti:
  filtro=np.logical_not(np.isin(saldos.ZONA,np.unique(txt.ZONA)))
  saldos=saldos[filtro]
  print(len(saldos)-filtro.sum(),'linhas excluídas de',len(saldos))

In [ ]:
if tem_collecti:
  saldos=pd.concat([txt,saldos],join='inner')

###Tratando tabela de Descuentos se houver e Substituindo Deudas com Desconto

In [ ]:
if tem_descuentos:
  #Deletando colunas indesejadas, agrupando por TERRITORIO e pegando o valor com menor desconto se houver duplicado
  coluna_importe='Monto a cancelar hasta el 30/09'
  colunas=['TERRITORIO',coluna_importe]
  descuentos=descuentos.sort_values('% DESCUENTO').groupby('TERRITORIO',as_index=False).first()[colunas]
  descuentos['TERRITORIO']=descuentos['TERRITORIO'].astype(str)
  #Merge com tabela principal
  saldos=saldos.merge(descuentos,on='TERRITORIO',how='left')
  #Renomeando coluna Importe por DEUDA A PAGAR
  saldos=saldos.rename(columns={coluna_importe:'DEUDA A PAGAR'})
  #Preenchendo DEUDA A PAGAR vazios POR DEU+PER
  filtro=saldos['DEUDA A PAGAR'].isna()
  saldos.loc[filtro,'DEUDA A PAGAR']=saldos.loc[filtro,'DEU+PER']
else:
  saldos['DEUDA A PAGAR']=saldos['DEU+PER']

###Tratamento da Carteira

####Apagar linhas nulas

In [ ]:
#Apagando linhas nulas
saldos.dropna(subset='TERRITORIO',inplace=True)

####Aplicando strip em todas as colunas

In [ ]:
#Aplicando strip em todas as colunas e substituindo } por '' e substituindo nan por ''
for coluna in saldos.columns.to_list():
  saldos[coluna]=saldos[coluna].astype(str)
  saldos[coluna]=saldos[coluna].str.strip().str.replace('}','')
  saldos[coluna]=saldos[coluna].str.replace('nan','')

<ipython-input-85-92f10093bc77>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  saldos[coluna]=saldos[coluna].str.strip().str.replace('}','')


####Formatando as colunas numéricas apra 2 casas

In [ ]:
#Formatando colunas numéricas para 2 casas decimais
colunas=['DEUDA INICIAL','DEUDA','DEU+PER','DEUDA A PAGAR']
for coluna in colunas:
  #Transformar vazios em 0
  saldos[coluna]=saldos[coluna].apply(lambda x: '0' if str(x)=='' else x)
  #Se último caractere for letra, retirar ele
  saldos[coluna]=saldos[coluna].apply(lambda x: str(x)[:-1] if str(x)[-1].isalpha() else str(x))
  #Formatando em número de 2 casas decimais
  saldos[coluna]=pd.to_numeric(saldos[coluna])
  saldos[coluna].fillna(0,inplace=True)
  saldos[coluna]=saldos[coluna].map('{:.2f}'.format)
  saldos[coluna]=saldos[coluna].astype(object)

###Tratamento de Número de Celular

In [ ]:
import re
colunas=['TELEFONO 1', 'TELEFONO 2', 'TELEFONO 3', 'TELEFONO 4']
#Tirar espaços
for coluna in colunas:
  #Tirar caracteres não numéricos
  saldos[coluna]=saldos[coluna].apply(lambda x: re.sub('[^0-9]', '', str(x)))
  #Tirar números com mais de 12 dígitos ou menos de 8 dígitos
  saldos[coluna]=saldos[coluna].apply(lambda x: '' if len(x)>12 or len(x)<8 else x)
  #Pegar últimos 9 dígitos de números com mais de 9 dígitos e completar com dígito 9
  saldos[coluna]=saldos[coluna].apply(lambda x: x[-9:] if len(x)>9 else ('9'+ x if len(x)==8 else x))
  #Apagar números que não começam com 9
  filtro=saldos[coluna]!=''
  saldos.loc[filtro,coluna]=saldos.loc[filtro,coluna].apply(lambda x: '' if x[0]!='9' else x)
#Completar TELEFONO 1 nulos pelo 2, 3 e 4
filtro=saldos['TELEFONO 1']==''
saldos.loc[filtro,'TELEFONO 1']=saldos.loc[filtro,'TELEFONO 2']
filtro=saldos['TELEFONO 1']==''
saldos.loc[filtro,'TELEFONO 1']=saldos.loc[filtro,'TELEFONO 3']
filtro=saldos['TELEFONO 1']==''
saldos.loc[filtro,'TELEFONO 1']=saldos.loc[filtro,'TELEFONO 4']

###Tratando as colunas ZONA, CARTA, SEC, EQUIPO, TERRITORIO de emails e saldos

In [ ]:
colunas=['ZONA', 'CARTA', 'SEC', 'EQUIPO', 'TERRITORIO']
for coluna in colunas:
    saldos[coluna]=saldos[coluna].astype(str).str.strip()
    emails[coluna]=emails[coluna].astype(str).str.strip()
saldos['EXTERNAL']=saldos['ZONA']+' '+saldos['CARTA']+' '+saldos['SEC']+' '+saldos['EQUIPO']+' '+saldos['TERRITORIO']
saldos['EXTERNAL']=saldos['EXTERNAL'].astype(str).str.strip()
emails['EXTERNAL']=emails['ZONA']+' '+emails['CARTA']+' '+emails['SEC']+' '+emails['EQUIPO']+' '+emails['TERRITORIO']
emails['EXTERNAL']=emails['EXTERNAL'].astype(str).str.strip()
emails.drop(colunas,axis=1,inplace=True)

###Mesclando coluna de Email

In [ ]:
print('Antes',len(saldos))
saldos=saldos.merge(emails,on='EXTERNAL',how='left')
print('Depois',len(saldos))

Antes 66237
Depois 66237


###Tratamento de Email

In [ ]:
#Aplicando lower na coluna de email
coluna_email='E-MAIL'
saldos[coluna_email]=saldos[coluna_email].str.lower().str.strip()
#Apagando emails não validados
saldos[coluna_email].loc[saldos[coluna_email].str.find('@')<0]=''
saldos['num @']=saldos[coluna_email].str.len()-saldos[coluna_email].str.replace('@','').str.len()
filtro=saldos['num @']!=1
saldos.loc[filtro,coluna_email]=''
saldos.drop('num @',axis=1,inplace=True)

###Preenchendo ULT.A#O FACTURACION Vazio


In [ ]:
saldos.loc[saldos['ULT.A#O FACTURACION']=='','ULT.A#O FACTURACION']='2018'

###Tratar coluna de Nome

In [ ]:
coluna_nomes=['APELLIDOS','NOMBRES']
for coluna_nome in coluna_nomes:
  saldos[coluna_nome]=saldos[coluna_nome].str.strip().str.title()

##Exportando

In [ ]:
#Carteira Tratada
saldos.to_csv(csv_saida,index=False)
if tem_collecti:
  #Arquivo de Emails.csv
  novo.to_csv(arquivo_emails,index=False)

##Movendo Arquivos da Ingestion

In [ ]:
if mover:
  if tem_collecti:
    arquivos=[nome_txt,nome_saldos]
  else:
     arquivos=[nome_saldos]
  for arquivo in arquivos:
    try:
      os.rename(ingestion + arquivo, processedfiles + arquivo)
    except:
      os.remove(processedfiles + arquivo)
      os.rename(ingestion + arquivo, processedfiles + arquivo)

# **Tratamento PRONTO**

## Inicialização

In [ ]:
mover=True

In [ ]:
import pandas as pd
import numpy as np
import os
import datetime
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
carteira='Pronto'

#Definindo diretórios
diretorio_tratamento=r'/content/drive/MyDrive/Processamentos/Tratamento/'
diretorio=diretorio_tratamento + carteira + '/'
output='/'.join(['/content/drive/MyDrive/Pronto pra upload',str(datetime.date.today().year),str(datetime.date.today().month).zfill(2),datetime.date.today().strftime('%d-%m-%Y')])+'/'
os.makedirs(output,exist_ok=True)
ingestion=diretorio+'ingestion/'
processedfiles=diretorio+'processedfiles/'

#Nome do arquivo de saída com caminho
csv_saida=output+'early_pronto-carteira-1_'+str(datetime.date.today().strftime('%d-%m-%Y'))+'.csv'

In [ ]:
#Esperando ler pelo menos 1 arquivo na ingestion
while len(os.listdir(ingestion))==0:
  time.sleep(5)
print(os.listdir(ingestion))

['Asignacion20231016.xlsx']


In [ ]:
#Identificando a planilha Asignacion
for arquivo in os.listdir(ingestion):
    if arquivo.find('Asignacion')>=0:
        nome_asignacion=arquivo

arquivo_asignacion=ingestion + nome_asignacion

In [ ]:
#Carregando Asignacion
df_asignacion=pd.read_excel(arquivo_asignacion,dtype=object)

## Início das Transformações

### Apagar coluna de índice

In [ ]:
#Criando cópia do arquivo principal
asignacion=df_asignacion.copy()
print(asignacion.columns)
print(asignacion.info())

#Deletando coluna de índices
del asignacion[asignacion.columns[0]]
len(asignacion.columns)

Index(['Unnamed: 0', 'Cedula', 'Cliente', 'cartera', 'Apellido1', 'Apellido2',
       'Nombre1', 'Nombre2', 'Edad', 'Sexo', 'CiuNom', 'DepNom', 'CPostal',
       'Producto', 'Numero', 'vencimiento', 'DeudaCliente', 'DeudaMin',
       'celular', 'email'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4987 entries, 0 to 4986
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    4987 non-null   object
 1   Cedula        4987 non-null   object
 2   Cliente       4987 non-null   object
 3   cartera       4987 non-null   object
 4   Apellido1     4987 non-null   object
 5   Apellido2     4767 non-null   object
 6   Nombre1       4987 non-null   object
 7   Nombre2       4374 non-null   object
 8   Edad          4987 non-null   object
 9   Sexo          4987 non-null   object
 10  CiuNom        4909 non-null   object
 11  DepNom        4920 non-null   object
 12  CPostal       4073 non-

19

In [ ]:
#Converter data pro formato brasileiro (não precisa)
'''asignacion['vencimiento']=pd.to_datetime(asignacion['vencimiento'])
asignacion['vencimiento']=asignacion['vencimiento'].dt.strftime('%d/%m/%Y')
asignacion['vencimiento']'''

"asignacion['vencimiento']=pd.to_datetime(asignacion['vencimiento'])\nasignacion['vencimiento']=asignacion['vencimiento'].dt.strftime('%d/%m/%Y')\nasignacion['vencimiento']"

### Adicionando as colunas: 'Moneda', 'Cuot minima deuda cliente' e 'Cantidad maxima cuotas' e reordenando as colunas

In [ ]:
#Adicionando colunas 'Moneda', 'Cuot minima deuda cliente' e 'Cantidad maxima cuotas' e reordenando as colunas
asignacion['Moneda']='UYU'
asignacion['Cuota minima deuda cliente']=200
asignacion['Cantidad maxima cuotas']=1
asignacion=pd.concat([asignacion.loc[:,:'CPostal'],asignacion.loc[:,'Moneda':'Cantidad maxima cuotas'],asignacion.loc[:,'Producto':'email']],join='outer',axis=1)
asignacion.columns

Index(['Cedula', 'Cliente', 'cartera', 'Apellido1', 'Apellido2', 'Nombre1',
       'Nombre2', 'Edad', 'Sexo', 'CiuNom', 'DepNom', 'CPostal', 'Moneda',
       'Cuota minima deuda cliente', 'Cantidad maxima cuotas', 'Producto',
       'Numero', 'vencimiento', 'DeudaCliente', 'DeudaMin', 'celular',
       'email'],
      dtype='object')

### Formatando 'DeudaCliente','DeudaMin' para 2 casas decimais

In [ ]:
#Formatando números para 2 casas decimais
colunas=['DeudaCliente','DeudaMin']
asignacion[colunas]=asignacion[colunas].astype(float)
asignacion[colunas]=asignacion[colunas].round(2)
for coluna in colunas:
  asignacion[coluna]=asignacion[coluna].map('{:.2f}'.format)
asignacion[colunas]

,DeudaCliente,DeudaMin
0,3194.73,3194.73
1,25109.92,6960.81
2,14536.29,7364.36
3,2301.84,2301.84
4,9165.51,9165.51
...,...,...
4982,44974.82,44974.82
4983,29434.40,15102.40
4984,13443.80,450.89
4985,12861.05,12861.05


### Retirando - e . da coluna Cedula

In [ ]:
#Retirando - e . da coluna Cedula
asignacion['Cedula']=asignacion['Cedula'].str.replace('.','').str.replace('-','')
asignacion['Cedula']

<ipython-input-106-ef483c74cf5f>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  asignacion['Cedula']=asignacion['Cedula'].str.replace('.','').str.replace('-','')


0       43212345
1       42459445
2       19452337
3       34935061
4       42310570
          ...   
4982    50157102
4983    50157102
4984    52570865
4985    34905335
4986    34905335
Name: Cedula, Length: 4987, dtype: object

###Renomeando as colunas

In [ ]:
asignacion.columns=['Cedula','Cliente','cartera','Apellido1','Apellido2','Nombre1','Nombre2',\
                    'Edad','Sexo','CiuNom','DepNom','CPostal','Moneda','Cuota minima deuda cliente',\
                    'Cantidad maxima cuotas','Producto','numero','Vencimiento','Deudacliente','Deudamin','Celular','email']

###Tratamento de Email

In [ ]:
#Aplicando lower e strip na coluna de email
coluna_email='email'
asignacion[coluna_email]=asignacion[coluna_email].str.lower().str.strip()
#Apagando emails não validados
asignacion.loc[asignacion[coluna_email].str.find('@')<0,coluna_email]=''
asignacion['num @']=asignacion[coluna_email].str.len()-asignacion[coluna_email].str.replace('@','').str.len()
filtro=asignacion['num @']!=1
asignacion.loc[filtro,coluna_email]=''
asignacion.drop('num @',axis=1,inplace=True)

###Tratamento do Celular (tirar 0 à esquerda)


In [ ]:
asignacion['Celular']=asignacion['Celular'].astype(int)

###Tratar coluna de Nome

In [ ]:
coluna_nomes=['Apellido1','Apellido2','Nombre1','Nombre2']
for coluna_nome in coluna_nomes:
  asignacion[coluna_nome]=asignacion[coluna_nome].str.strip().str.title()

## Exportando

In [ ]:
asignacion.to_csv(csv_saida,index=False)

## Movendo arquivos processados

In [ ]:
if mover:
  arquivos=[nome_asignacion]
  for arquivo in arquivos:
    try:
      os.rename(ingestion + arquivo, processedfiles + arquivo)
    except:
      os.remove(processedfiles + arquivo)
      os.rename(ingestion + arquivo, processedfiles + arquivo)

# **Tratamento PORTO ESSENCIAL**

In [ ]:
import pandas as pd
import numpy as np
import datetime
import os
import time

##Mover arquivos ao finalizar o tratamento?

In [ ]:
mover=True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
carteira='Porto Essencial'

#Definindo diretórios
diretorio_tratamento=r'/content/drive/MyDrive/Processamentos/Tratamento/'
diretorio=diretorio_tratamento + carteira + '/'
output='/'.join(['/content/drive/MyDrive/Pronto pra upload',str(datetime.date.today().year),str(datetime.date.today().month).zfill(2),datetime.date.today().strftime('%d-%m-%Y')])+'/'
os.makedirs(output,exist_ok=True)
processedfiles=diretorio+'processedfiles/'
ingestion=diretorio+'ingestion/'

#Nome do arquivo de saída com caminho
csv_saida=output+'early_porto-essencial_'+str(datetime.date.today().strftime('%d-%m-%Y'))+'.csv'

In [ ]:
#Esperando ler 2 arquivos na ingestion
while len(os.listdir(ingestion))!=2:
  time.sleep(5)
print(os.listdir(ingestion))

['ESC_ESSEN_565.CSV', 'TABELA_ESSENCIAL_HISOFI.xlsx']


In [ ]:
#Lendo os 2 arquivos
for arquivo in os.listdir(ingestion):
  if arquivo.find('ESC_ESSEN')>=0:
    print('ESC_ESSEN:',arquivo)
    nome_essen=arquivo
    esc_essen=pd.read_csv(ingestion+arquivo,dtype=object)
  elif arquivo.find('TABELA_ESSENCIAL')>=0:
    nome_tab_essen=arquivo
    tab_essen=pd.read_excel(ingestion+arquivo,dtype=object)
    print('TABELA_ESSENCIAL:',arquivo)

ESC_ESSEN: ESC_ESSEN_565.CSV
TABELA_ESSENCIAL: TABELA_ESSENCIAL_HISOFI.xlsx


##Início dos Tratamentos

### Tratamento de TABELA_ESSENCIAL

In [ ]:
#Listando colunas repetidas da tabela auxiliar diferente de NR_RAS
col_repetida=[]
for coluna in tab_essen.columns.to_list():
    if coluna in esc_essen.columns.to_list() and coluna!='NR_RAS':
        col_repetida.append(coluna)

print('Colunas repetidas:') #Resultados anteriores: ['DT_FECH_SINISTRO', 'VR_DESPESAS_SIN']
print(col_repetida)

Colunas repetidas:
['DT_FECH_SINISTRO', 'VR_DESPESAS_SIN']


In [ ]:
#Capitalizando nomes
tab_essen['NM_GARANTIDO']=tab_essen['NM_GARANTIDO'].str.title()

In [ ]:
#Excluindo colunas repetidas de tab_essen
tab_essen.drop(col_repetida,axis=1,inplace=True)

In [ ]:
#Tratando '..' na coluna NR_CELULAR_PORTO de tab_essen
tab_essen['NR_CELULAR_PORTO']=tab_essen['NR_CELULAR_PORTO'].apply(lambda x: '' if str(x)=='..' else x)

In [ ]:
#Excluindo colunas indesejadas
col_final=['NR_RAS','NR_COD_ESCRITORIO_NOVO','DT_FECH_SINISTRO','QT_DIAS_ATRASO','VR_DESPESAS_SIN','VR_EM_ATRASO_COM_JUROS','PC_DES_AVISTA',\
                    'PC_DES_2_3_VEZES','PC_DES_4_6_VEZES','PC_DES_7_12_VEZES','PC_DES_13_20_VEZES','PC_DESC_20_VEZES','VR_VALOR_COM_DESPESA','PROPENSAO',\
                    'NM_GARANTIDO','NR_CPF_V2','NR_CNPJ_V2','NR_CELULAR_PORTO','CD_UF','VR_COBERTURA_ALUGUEL','EMAIL_1','DT_NASCIMENTO_INQUILINO']
excluir=[]
for coluna in tab_essen.columns.to_list():
  if coluna not in col_final:
    excluir.append(coluna)
tab_essen=tab_essen.drop(excluir, axis=1)

In [ ]:
#Ordenando por email e agrupando por NR_RAS
tab_essen=tab_essen.sort_values('EMAIL_1',ascending=False)
tab_essen=tab_essen.groupby('NR_RAS',as_index=False).first()

In [ ]:
#Convertendo datas pro formato brasileiro
for coluna in tab_essen.select_dtypes(include='datetime').columns.tolist():
    tab_essen[coluna]=tab_essen[coluna].apply(lambda x: str(x)[:10])
    tab_essen[coluna]=pd.to_datetime(tab_essen[coluna])
    tab_essen[coluna]=tab_essen[coluna].dt.strftime('%d/%m/%Y')

### Tratamento de ESC_ESSEN

In [ ]:
#Tratando com separador de milhares como '.' e de decimal como ','
tratar_ponto_virgula=['VR_EM_ATRASO_COM_JUROS','VR_VALOR_COM_DESPESA']

for coluna in tratar_ponto_virgula:
  esc_essen[coluna]=esc_essen[coluna].apply(lambda x: str(x).replace('.',''))
  esc_essen[coluna]=esc_essen[coluna].apply(lambda x: str(x).replace(',','.'))
  esc_essen[coluna]=esc_essen[coluna]=esc_essen[coluna]=esc_essen[coluna].astype(float)
  print(esc_essen[coluna])

0        1187.03
1        4543.49
2        3022.40
3        1762.00
4        4396.88
          ...   
11863    1505.24
11864    5209.03
11865    4742.92
11866    4319.42
11867    1422.64
Name: VR_EM_ATRASO_COM_JUROS, Length: 11868, dtype: float64
0         944.71
1        3569.19
2        2405.40
3        1402.30
4        3499.29
          ...   
11863    1500.00
11864    5190.89
11865    4729.70
11866    4311.39
11867    1420.00
Name: VR_VALOR_COM_DESPESA, Length: 11868, dtype: float64


### Mesclando as duas tabelas

In [ ]:
#Fazendo o merge das Tabelas
merged=esc_essen.merge(tab_essen,on='NR_RAS',how='inner')

In [ ]:
#Eliminando colunas indesejadas
merged=merged[col_final]

###Tratando os números de telefone e email

In [ ]:
#Validação de Celular
col_celular='NR_CELULAR_PORTO'
filtro=np.char.str_len(np.array(merged[col_celular]).astype(str))!=11
merged.loc[filtro,col_celular]=''
#Aplicando lower e strip na coluna de email
coluna_email='EMAIL_1'
merged[coluna_email]=merged[coluna_email].str.lower().str.strip()
#Apagando emails não validados
merged[coluna_email].loc[merged[coluna_email].str.find('@')<0]=''
merged['num @']=merged[coluna_email].str.len()-merged[coluna_email].str.replace('@','').str.len()
filtro=merged['num @']!=1
merged.loc[filtro,coluna_email]=''
merged.drop('num @',axis=1,inplace=True)

<ipython-input-129-371e18857a3c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged[coluna_email].loc[merged[coluna_email].str.find('@')<0]=''


###Tratar coluna de Nome

In [ ]:
coluna_nomes=['NM_GARANTIDO']
for coluna_nome in coluna_nomes:
  merged[coluna_nome]=merged[coluna_nome].str.strip().str.title()

##Exportando

In [ ]:
#Exportando o arquivo para Output
merged.to_csv(csv_saida,index=False)

##Movendo arquivos

In [ ]:
#Movendo arquivos para processedfiles
if mover:
  arquivos=[nome_tab_essen,nome_essen]
  for arquivo in arquivos:
    try:
      os.rename(ingestion + arquivo, processedfiles + arquivo)
    except:
      os.remove(processedfiles + arquivo)
      os.rename(ingestion + arquivo, processedfiles + arquivo)

# **Tratamento RISEL**

## Inicialização

In [ ]:
mover=True

In [ ]:
import pandas as pd
import numpy as np
import os
import datetime
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
carteira='Risel'

#Definindo diretórios
diretorio_tratamento=r'/content/drive/MyDrive/Processamentos/Tratamento/'
diretorio=diretorio_tratamento + carteira + '/'
output='/'.join(['/content/drive/MyDrive/Pronto pra upload',str(datetime.date.today().year),str(datetime.date.today().month).zfill(2),datetime.date.today().strftime('%d-%m-%Y')])+'/'
ingestion=diretorio+'ingestion/'
processedfiles=diretorio+'processedfiles/'
os.makedirs(output,exist_ok=True)
os.makedirs(ingestion,exist_ok=True)
os.makedirs(processedfiles,exist_ok=True)

today=datetime.date.today()
#Nome do arquivo de saída com caminho
csv_saida=output+'early_risel_'+str(today.strftime('%d-%m-%Y'))+'.csv'

In [ ]:
#Esperando ler 1
num_arquivo=1

while len(os.listdir(ingestion))!=num_arquivo:
  time.sleep(5)
print(os.listdir(ingestion))

['1691077668699template_sofi.xlsx']


In [ ]:
#Identificando as planilhas EXCLUSIVA e ESPECIAL
for arquivo in os.listdir(ingestion):
    if arquivo.find('template_sofi')>=0:
      nome_exclusiva=arquivo
      arquivo_exclusiva=ingestion+arquivo

In [ ]:
#Carregando Planilhas
df_endereco=pd.read_excel(arquivo_exclusiva,dtype=object,sheet_name='ENDERECO')
df_titulos=pd.read_excel(arquivo_exclusiva,dtype=object,sheet_name='TITULOS')

In [ ]:
#Criar cópias
endereco=df_endereco.copy()
titulos=df_titulos.copy()

## Início das Transformações

###Tratar Tabela de ENDEREÇO

####Excluir colunas repetidas da tabela ENDEREÇO

In [ ]:
excluir=[]
for coluna in endereco.columns.to_list():
  if coluna in titulos.columns.to_list():
    excluir.append(coluna)
endereco.drop(excluir, axis=1, inplace=True)

####Agrupar por COD

In [ ]:
endereco=endereco.groupby('COD',as_index=False).last()

###Mesclar tabelas

In [ ]:
merged=titulos.merge(endereco,left_on='CODCLIENTE',right_on='COD',how='left')
merged.drop('COD', axis=1, inplace=True)

###Tratamento dos números de telefone

In [ ]:
merged['TELEFONE']=merged['TELEFONE'].str.replace(' ','').str.replace('-','').str.replace(')','').str.replace('(','')

<ipython-input-144-aff8f3d8da88>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  merged['TELEFONE']=merged['TELEFONE'].str.replace(' ','').str.replace('-','').str.replace(')','').str.replace('(','')


###Tratamento de Endereço

In [ ]:
#Preenchendo vazios
colunas=['ENDERECO','BAIRRO','NUMEND','COMPLMENTO']
for coluna in colunas:
  merged[coluna]=merged[coluna].fillna('')
#Concatenando endereço
merged['ENDERECO COMPLETO']=merged['ENDERECO'] + ' ' + merged['BAIRRO'] + ' ' + merged['NUMEND'] + ' ' + merged['COMPLMENTO']
merged.reset_index(drop=True,inplace=True)
#Remover Espaços duplos ou superiores
for i in range(len(merged)):
  split=merged.loc[i,'ENDERECO COMPLETO'].split(' ')  #aplicar split por ' '
  split=[s for s in split if s!='']  #remover strings vazias do array
  merged.loc[i,'ENDERECO COMPLETO']=' '.join(split) #juntar strings de split sem os ''

###Adicionar coluna de Payload

In [ ]:
merged['payload']=merged['FILIAL'].astype(str) + ';' + merged['ESTAB'].astype(str)

###Tratamento de Email

In [ ]:
#Escolher o primeiro email sem @risel
#Split em email
merged['EMAIL']=merged['EMAIL'].str.split(';')
#Flatten em email
merged=merged.explode('EMAIL')
#Apagar emails com @risel
filtro=merged['EMAIL'].str.find('@risel')>=0
merged.loc[filtro,'EMAIL']=''
#Ordenar por email desc
merged=merged.sort_values('EMAIL',ascending=False)
#Manter o primeiro email
merged=merged.drop_duplicates('DOC',keep='first')

In [ ]:
# Aplicando lower e strip na coluna de email
coluna_email = 'EMAIL'
merged[coluna_email] = merged[coluna_email].str.lower().str.strip()
# Apagando emails não validados
merged[coluna_email].loc[merged[coluna_email].str.find('@') < 0] = ''
merged['num @'] = merged[coluna_email].str.len() - merged[coluna_email].str.replace('@', '').str.len()
filtro = merged['num @'] != 1
merged.loc[filtro, coluna_email] = ''
merged.drop('num @', axis=1, inplace=True)

###Tratar coluna de Nome

In [ ]:
coluna_nomes=['CLIENTE']
for coluna_nome in coluna_nomes:
  merged[coluna_nome]=merged[coluna_nome].str.strip().str.title()

###Renomear Colunas

In [ ]:
merged=merged.rename(columns={'DOC':'external_id', 'CNPJ':'document', 'CLIENTE':'full_name', 'TELEFONE':'phone', \
                       'EMAIL':'email', 'VALOR':'amount', 'CODCLIENTE':'product',\
                       'VENCIMENTO':'due_date', 'ENDERECO COMPLETO':'address', 'CEP':'zipcode', 'CIDADE':'city', 'UF':'state',\
                       'EMISSAO':'since', 'payload':'payload'})

## Exportando

In [ ]:
merged.to_csv(csv_saida,index=False)

## Movendo Somente as Carteiras Exclusivas

In [ ]:
if mover:
  arquivos=[nome_exclusiva]
  for arquivo in arquivos:
    try:
      os.rename(ingestion + arquivo, processedfiles + arquivo)
    except:
      os.remove(processedfiles + arquivo)
      os.rename(ingestion + arquivo, processedfiles + arquivo)

#**Tratamento Porto Batimento (Segunda)**

In [ ]:
import pandas as pd
import numpy as np
import datetime
import os
import time
import datetime
import sys

In [ ]:
today=datetime.date.today()
#Checando se é segunda
if today.weekday()!=0:
  print('Hoje NÃO é segunda! Rodar outro script!')
  sys.exit()

##Mover arquivos ao finalizar o tratamento?

In [ ]:
mover=True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
carteira='Porto Batimento'

#Definindo diretórios
diretorio_tratamento=r'/content/drive/MyDrive/Processamentos/Tratamento/'
diretorio=diretorio_tratamento + carteira + '/'
output='/'.join(['/content/drive/MyDrive/Pronto pra upload',str(datetime.date.today().year),str(datetime.date.today().month).zfill(2),datetime.date.today().strftime('%d-%m-%Y')])+'/'
os.makedirs(output,exist_ok=True)
processedfiles=diretorio+'processedfiles/'
ingestion=diretorio+'ingestion/'
auxiliary=diretorio+'auxiliary/'
#Nome do arquivo de saída com caminho
csv_saida=output+'early_porto-batimento_'+str(datetime.date.today().strftime('%d-%m-%Y'))+'.csv'

In [ ]:
#Esperando ler n arquivos na ingestion
n=7

while len(os.listdir(ingestion))!=n:
  time.sleep(5)
print(os.listdir(ingestion))

['ESC_565.CSV', 'TB_BATIMENTO_HISOFI4.xlsx', 'TB_BATIMENTO_HISOFI3.xlsx', 'TB_BATIMENTO_HISOFI2.xlsx', 'TB_BATIMENTO_HISOFI1.xlsx', 'TB_BATIMENTO_HISOFI.xlsx', 'TB_DIST_HISOFI.xlsx']


In [ ]:
#Lendo os 7 arquivos (6 tabelas e 1 csv)
i=1
for arquivo in os.listdir(ingestion):
  if arquivo.find('TB')>=0:
    if i==1:
      df=pd.read_excel(ingestion+arquivo,dtype=object)
      print(arquivo,'importado com sucesso!')
      i+=1
    else:
      df2=pd.read_excel(ingestion+arquivo,dtype=object)
      df=pd.concat([df,df2],join='outer',ignore_index=False)
      print(arquivo,'concatenado com sucesso!')
  elif arquivo.find('ESC')>=0:
    df_esc=pd.read_csv(ingestion+arquivo,dtype=object)
    print(arquivo,'importado com sucesso!')

ESC_565.CSV importado com sucesso!
TB_BATIMENTO_HISOFI4.xlsx importado com sucesso!
TB_BATIMENTO_HISOFI3.xlsx concatenado com sucesso!
TB_BATIMENTO_HISOFI2.xlsx concatenado com sucesso!
TB_BATIMENTO_HISOFI1.xlsx concatenado com sucesso!
TB_BATIMENTO_HISOFI.xlsx concatenado com sucesso!
TB_DIST_HISOFI.xlsx concatenado com sucesso!


In [ ]:
#Criando cópia dos arquivos importados para fazer alteração
concatenado=df.copy()
esc=df_esc.copy()

In [ ]:
col_final=['NR_RAS','NR_COD_ESCRITORIO_NOVO','DT_FECH_SINISTRO','QT_DIAS_ATRASO','VR_DESPESAS_SIN','VR_EM_ATRASO_COM_JUROS','PC_DES_AVISTA','PC_DES_2_3_VEZES','PC_DES_4_6_VEZES',\
           'PC_DES_7_12_VEZES','PC_DES_13_20_VEZES','PC_DESC_20_VEZES','VR_VALOR_COM_DESPESA','PROPENSAO','NM_GARANTIDO','NR_CPF_V2','NR_CNPJ_V2','NR_CELULAR_PORTO','CD_UF','VR_COBERTURA_ALUGUEL',\
           'EMAIL_1','DT_NASCIMENTO_INQUILINO']

##Início dos Tratamentos

### Tratamento das Tabelas Auxiliares

In [ ]:
#Ordenando por Email e Mantendo o primeiro
concatenado=concatenado.sort_values('EMAIL_1',ascending=False)
concatenado=concatenado.drop_duplicates('NR_RAS',keep='first')

In [ ]:
#Listando colunas repetidas da tabela auxiliar diferente de NR_RAS
col_repetida=[]
for coluna in concatenado.columns.to_list():
    if coluna in esc.columns.to_list() and coluna!='NR_RAS':
        col_repetida.append(coluna)

print('Colunas repetidas:') #Resultados anteriores: ['DT_FECH_SINISTRO', 'VR_DESPESAS_SIN']
print(col_repetida)
#Excluindo colunas repetidas de concatenado
concatenado.drop(col_repetida,axis=1,inplace=True)

Colunas repetidas:
['DT_FECH_SINISTRO', 'VR_DESPESAS_SIN']


In [ ]:
#Excluindo colunas indesejadas
excluir=[]
for coluna in concatenado.columns.to_list():
  if coluna not in col_final:
    excluir.append(coluna)
concatenado=concatenado.drop(excluir, axis=1)
concatenado

,DT_NASCIMENTO_INQUILINO,NR_RAS,NM_GARANTIDO,CD_UF,NR_CELULAR_PORTO,VR_COBERTURA_ALUGUEL,NR_CPF_V2,NR_CNPJ_V2,EMAIL_1
8782,NaN,0746 2017 011627,NILCIANE COUTINHO DOS SANTOS,MG,31992865111,17389.2,01725097699,NaN,zuretacici@gmail.com
10824,NaN,0746 2018 005150,JULIANA DOS SANTOS FERREIRA,SP,11976940280,70500,30695815873,NaN,zulinqui22@gmail.com
11531,NaN,0746 2015 018298,ZULEMA SAVINO SANTANA,SP,11976340438,36000,25732148850,NaN,zulemassantana@gmail.com
5115,NaN,0746 2014 012711,ZULEICA SABINO PIRES,SP,11981305604,45360,33605693857,NaN,zuleica_oliveira@hotmail.com
4494,NaN,0746 2014 010059,ZAMIR POMPEO TEIXEIRA,RS,51995496688,43335,17901464020,NaN,zspt@pop.com.br
...,...,...,...,...,...,...,...,...,...
11743,NaN,0746 2018 014722,SERGIO BESSA PINTO,RJ,21997873236,38589,43714625704,NaN,NaN
11744,NaN,0746 2018 014840,DIEFERSON CASTILHO FRANCISCO,PR,41995475622,17250,06827739996,NaN,NaN
11753,NaN,0746 2018 015037,DINEI MATTOS DE OLIVEIRA,SP,13997346401,33168.9,13388233888,NaN,NaN
11754,NaN,0746 2018 015054,ELIZABETH PERUCCI DIAS,SP,11976451861,33000,94636265815,NaN,NaN


In [ ]:
#Capitalizando nomes
concatenado['NM_GARANTIDO']=concatenado['NM_GARANTIDO'].str.title()

In [ ]:
#Tratando '..' na coluna NR_CELULAR_PORTO de concatenado
concatenado['NR_CELULAR_PORTO']=concatenado['NR_CELULAR_PORTO'].apply(lambda x: '' if str(x)=='..' else str(x))

In [ ]:
concatenado['VR_COBERTURA_ALUGUEL']=concatenado['VR_COBERTURA_ALUGUEL'].astype(float)
concatenado['VR_COBERTURA_ALUGUEL']=concatenado['VR_COBERTURA_ALUGUEL'].round(2)
concatenado['VR_COBERTURA_ALUGUEL']=concatenado['VR_COBERTURA_ALUGUEL'].map('{:.2f}'.format)

### Tratamento de ESC

In [ ]:
#Excluindo colunas indesejadas
excluir=[]
for coluna in esc.columns.to_list():
  if coluna not in col_final:
    excluir.append(coluna)
esc=esc.drop(excluir, axis=1)
esc.head()

,NR_RAS,NR_COD_ESCRITORIO_NOVO,DT_FECH_SINISTRO,QT_DIAS_ATRASO,VR_DESPESAS_SIN,VR_EM_ATRASO_COM_JUROS,PC_DES_AVISTA,PC_DES_2_3_VEZES,PC_DES_4_6_VEZES,PC_DES_7_12_VEZES,PC_DES_13_20_VEZES,PC_DESC_20_VEZES,VR_VALOR_COM_DESPESA,PROPENSAO
0,0746 2003 002317,565,29/03/2004,7145,"0,00","14.986,40",99.33%,99.33%,99.10%,94.38%,89.89%,85.61%,"2.845,63",1-Altamente propenso
1,0746 2003 002515,565,22/09/2003,7334,"0,00","4.526,05",97.79%,97.79%,97.79%,97.79%,94.91%,90.39%,"822,46",1-Altamente propenso
2,0746 2003 002520,565,10/01/2005,6858,"1.068,05","45.199,79",99.78%,99.78%,99.78%,99.78%,99.78%,99.78%,"10.242,91",1-Altamente propenso
3,0746 2003 002530,565,03/09/2003,7353,"0,00","8.398,10",98.81%,98.81%,98.81%,98.67%,93.97%,89.50%,"1.519,35",1-Altamente propenso
4,0746 2003 002547,565,16/10/2003,7310,"0,00","17.332,53",99.42%,99.42%,99.42%,95.32%,90.78%,86.46%,"3.167,24",1-Altamente propenso


In [ ]:
#Tratando com separador de milhares como '.' e de decimal como ','
tratar_ponto_virgula=['VR_DESPESAS_SIN','VR_EM_ATRASO_COM_JUROS','VR_VALOR_COM_DESPESA']

for coluna in tratar_ponto_virgula:
  esc[coluna]=esc[coluna].apply(lambda x: str(x).replace('.',''))
  esc[coluna]=esc[coluna].apply(lambda x: str(x).replace(',','.'))
  esc[coluna]=esc[coluna]=esc[coluna]=esc[coluna].astype(float)
  print(esc[coluna])

0           0.00
1           0.00
2        1068.05
3           0.00
4           0.00
          ...   
46636     215.30
46637     151.30
46638     487.50
46639       0.00
46640     172.30
Name: VR_DESPESAS_SIN, Length: 46641, dtype: float64
0        14986.40
1         4526.05
2        45199.79
3         8398.10
4        17332.53
           ...   
46636    11191.40
46637     1319.99
46638     3316.48
46639     1977.86
46640     5089.01
Name: VR_EM_ATRASO_COM_JUROS, Length: 46641, dtype: float64
0         2845.63
1          822.46
2        10242.91
3         1519.35
4         3167.24
           ...   
46636     7575.92
46637     1019.46
46638     2668.76
46639     1300.54
46640     3517.80
Name: VR_VALOR_COM_DESPESA, Length: 46641, dtype: float64


### Mesclando as duas tabelas

In [ ]:
#Fazendo o merge das Tabelas
merged=esc.merge(concatenado,on='NR_RAS',how='inner')

In [ ]:
#Eliminando colunas indesejadas
merged=merged[col_final]

###Tratando os números de telefone e email

In [ ]:
#Validação de Celular
col_celular='NR_CELULAR_PORTO'
filtro=np.char.str_len(np.array(merged[col_celular]).astype(str))!=11
merged.loc[filtro,col_celular]=''

#Validação de Email
#Aplicando lower e strip na coluna de email
coluna_email='EMAIL_1'
merged[coluna_email]=merged[coluna_email].str.lower().str.strip()
#Apagando emails não validados
merged[coluna_email].loc[merged[coluna_email].str.find('@')<0]=''
merged['num @']=merged[coluna_email].str.len()-merged[coluna_email].str.replace('@','').str.len()
filtro=merged['num @']!=1
merged.loc[filtro,coluna_email]=''
merged.drop('num @',axis=1,inplace=True)

<ipython-input-172-4b88d990f8a8>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged[coluna_email].loc[merged[coluna_email].str.find('@')<0]=''


###Tratar coluna de Nome

In [ ]:
coluna_nomes=['NM_GARANTIDO']
for coluna_nome in coluna_nomes:
  merged[coluna_nome]=merged[coluna_nome].str.strip().str.title()

##Exportando

In [ ]:
#Exportando Tabela Auxiliar para auxiliary
concatenado.to_csv(auxiliary+'Concatenado.csv',index=False)
#Exportando o arquivo para Output
merged.to_csv(csv_saida,index=False)

##Movendo arquivos

In [ ]:
#Movendo arquivos para processedfiles
if mover:
  arquivos=os.listdir(ingestion)
  for arquivo in arquivos:
    try:
      os.rename(ingestion + arquivo, processedfiles + arquivo)
    except:
      os.remove(processedfiles + arquivo)
      os.rename(ingestion + arquivo, processedfiles + arquivo)

#**Pode rodar o código inteiro (já tem loop automático)**

##Abrir drive de relatórios assim que rodar o script pra baixar assignados e rep

#**Listar arquivos das ingestions**

In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
diretorio='/content/drive/MyDrive/Processamentos/Pagos/'

In [ ]:
os.listdir(diretorio)

['Natura MEX',
 'Pronto',
 '_Descontos',
 'Payments_done_avon_peru',
 'Avon Peru',
 'Payments_done_pronto',
 'Natura Peru',
 'Payments_done_natura_mex',
 '.ipynb_checkpoints',
 'Payments_done_natura_peru']

In [ ]:
ingestions=['Natura MEX/ingestion_natura-mex/',\
 'Pronto/ingestion_pronto/',\
 'Avon Peru/ingestion_avon-peru/',\
 'Natura Peru/ingestion_natura-peru/']
num_arquivo_ingestion=[]
for ingestion in ingestions:
  print(ingestion)
  print(os.listdir(diretorio+ingestion+'/'),'\n',len(os.listdir(diretorio+ingestion+'/')),'\n')
  num_arquivo_ingestion.append(len(os.listdir(diretorio+ingestion+'/')))

Natura MEX/ingestion_natura-mex/
['13-10-2023_Consulta_Q1390_P5115922_20231016052629350.csv', '14-10-2023_Consulta_Q1390_P5115922_20231016052834767.csv', '15-10-2023_Consulta_Q1390_P5115922_20231016052629063.csv'] 
 3 

Pronto/ingestion_pronto/
['13-10-2023_Pagos20231013.xlsx', '14-10-2023_Pagos20231014.xlsx'] 
 2 

Avon Peru/ingestion_avon-peru/
['13-10-23_PAGOS_CO.TXT'] 
 1 

Natura Peru/ingestion_natura-peru/
[] 
 0 



#**Pagos Pronto (com loop)**

In [ ]:
for i in range(num_arquivo_ingestion[1]):
  #**Pagos Pronto**

  import pandas as pd
  import numpy as np
  import os
  import time
  import datetime

  from google.colab import drive
  drive.mount('/content/drive')

  ##Definindo diretórios e nomes de arquivos

  #Diretório de saída do relatório
  carteira='Pronto'
  dir_pagos='/content/drive/MyDrive/Processamentos/Pagos/'
  dir_ingestion=dir_pagos+carteira+'/ingestion_pronto/'
  dir_processedfiles=dir_pagos+carteira+'/processedfiles/'
  dir_auxiliary=dir_pagos+carteira+'/auxiliary/'
  dir_saida=dir_pagos+'Payments_done_pronto/'
  diretorios=[dir_pagos,dir_ingestion,dir_processedfiles,dir_saida]
  for diretorio in diretorios:
    os.makedirs(diretorio,exist_ok=True)
  #Diretório dos arquivos tratados de hoje
  dir_tratados='/content/drive/MyDrive/Pronto pra upload/'

  #Procurando arquivo de Consulta na ingestion
  while len(os.listdir(dir_ingestion))==0:
    time.sleep(5)
  for arquivo in os.listdir(dir_ingestion):
    if arquivo.find('Pagos')>=0:
      nome_consulta=arquivo

  #Pegando Data da Consulta
  data_consulta=datetime.date(int(nome_consulta[6:10]),int(nome_consulta[3:5]),int(nome_consulta[:2]))

  #Validando Data da Consulta
  today=datetime.date.today()
  if today.weekday()==0:
    dias=3
  else:
    dias=1
  '''if today-data_consulta>pd.Timedelta(days=dias):
    sys.exit()'''

  #Gerando data da carteira
  '''if data_consulta.weekday()==0:
    dias=3
  elif data_consulta.weekday()==6:
    dias=2
  else:
    dias=1'''
  data_carteira=today

  #Nome dos arquivos de hoje e ontem
  nome_hoje='early_pronto-carteira-1_'+str(data_carteira.strftime('%d-%m-%Y'))+'.csv'
  dir_carteira=dir_tratados+str(data_carteira.year)+'/'+str(data_carteira.month).zfill(2)+'/'+str(data_carteira.strftime('%d-%m-%Y'))+'/'
  nome_carteiras='Carteiras.csv'

  #Nome excel de saída
  excel_saida=dir_saida + str(data_consulta.strftime('%d-%m-%Y')) + '_pagos_pronto-carteira-1'+'.xlsx'
  print(data_consulta)

  ##Importação dos arquivos

  df_consulta=pd.read_excel(dir_ingestion+nome_consulta,dtype=object)
  df_carteira=pd.read_csv(dir_carteira+nome_hoje,dtype=object)
  df_carteiras=pd.read_csv(dir_auxiliary+nome_carteiras,dtype=object)

  ##Concatenar Carteira de hoje em Carteiras

  df_carteiras=pd.concat([df_carteiras,df_carteira],join='inner')
  #Retirar duplicados em Cedula e numero
  df_carteiras.drop_duplicates(subset=['Cedula','numero'],inplace=True)

  ##Criação de cópias

  consulta=df_consulta.copy()
  carteiras=df_carteiras.copy()

  ##Tratamento

  ###Deletar coluna de índices

  consulta.drop('Unnamed: 0',axis=1,inplace=True)

  ###Mesclar CodigoPedido (external_id)

  #Tratar coluna Cedula
  consulta['Cedula']=consulta['Cedula'].str.replace('.','').str.replace('-','').str.strip()
  #Manter somente colunas necessárias na Carteira
  colunas=['Cedula','numero','Cliente']
  carteiras=carteiras[colunas]
  #Mesclando
  consulta=consulta.merge(carteiras, left_on=['Cedula','nro_prod'], right_on=['Cedula','numero'], how='left')
  #Criar coluna de External ID
  consulta['CodigoPedido']=consulta['Cliente']+' '+consulta['nro_prod']

  ###Renomear Colunas

  consulta=consulta.rename(columns={'pcgfechapago':'FechaPago','CodigoPedido':'CodigoPedido','pcgimportepagado':'ValorPagamento'})

  ###Formatar coluna de Pagamento

  consulta['ValorPagamento']=consulta['ValorPagamento'].astype(str)

  ###Remover Duplicados

  consulta.drop_duplicates(subset=consulta.columns,inplace=True)

  ###Adicionar índice

  consulta['ID']=consulta.index
  consulta['ID']=consulta['ID']+1
  consulta['ID']=consulta['ID'].astype(str).str.zfill(5)

  ##Exportando

  consulta.to_excel(excel_saida,index=False)
  carteiras.to_csv(dir_auxiliary+nome_carteiras,index=False)

  ##Movendo Arquivos

  os.rename(dir_ingestion+nome_consulta,dir_processedfiles+nome_consulta)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2023-10-18


<ipython-input-180-b030f1b545a4>:91: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  consulta['Cedula']=consulta['Cedula'].str.replace('.','').str.replace('-','').str.strip()


#**Pagos Avon Peru Padrão (com loop)**

In [ ]:
for i in range(num_arquivo_ingestion[2]):
  #**Pagos Avon Peru Padrão**

  import pandas as pd
  import numpy as np
  import os
  import time
  import datetime

  from google.colab import drive
  drive.mount('/content/drive')

  ##Definindo diretórios e nomes de arquivos

  #Diretório de saída do relatório
  carteira='Avon Peru'
  dir_pagos='/content/drive/MyDrive/Processamentos/Pagos/'
  dir_ingestion=dir_pagos+carteira+'/ingestion_avon-peru/'
  dir_processedfiles=dir_pagos+carteira+'/processedfiles/'
  dir_saida=dir_pagos+'Payments_done_avon_peru/'
  diretorios=[dir_pagos,dir_ingestion,dir_processedfiles,dir_saida]
  for diretorio in diretorios:
    os.makedirs(diretorio,exist_ok=True)
  #Diretório dos arquivos tratados de hoje
  dir_tratados='/content/drive/MyDrive/Pronto pra upload/'

  #Procurando arquivo de Consulta na ingestion
  while len(os.listdir(dir_ingestion))==0:
    time.sleep(5)
  for arquivo in os.listdir(dir_ingestion):
    nome_consulta=arquivo

  #Pegando Data da Consulta
  data_consulta=datetime.date(int(nome_consulta[6:10]),int(nome_consulta[3:5]),int(nome_consulta[:2]))

  #Validando Data da Consulta
  today=datetime.date.today()
  if today.weekday()==0:
    dias=3
  else:
    dias=1
  '''if today-data_consulta>pd.Timedelta(days=dias):
    sys.exit()'''

  #Nome excel de saída
  csv_saida=dir_saida + str(data_consulta.strftime('%d-%m-%Y')) + '_pagos_avon-pe-padrao'+'.csv'
  print(data_consulta)

  ##Importação dos arquivos

  df_consulta=pd.read_csv(dir_ingestion+nome_consulta,dtype=object,sep='\t')
  #df_consulta=pd.read_csv(dir_ingestion+nome_consulta,dtype=object,sep=',')

  ##Criação de cópias

  consulta=df_consulta.copy()

  ###Adicionar CodigoPedido

  colunas=['ZONA','SECC','EQUIPO','TERRITORIO','CARTA']
  for coluna in colunas:
    consulta[coluna]=consulta[coluna].str.strip()
  consulta['CodigoPedido']=consulta['ZONA']+' '+consulta['CARTA']+' '+consulta['SECC']+' '+consulta['EQUIPO']+ ' '+ consulta['TERRITORIO']
  consulta.drop(colunas, axis=1,inplace=True)

  ###Renomear Colunas

  consulta=consulta.rename(columns={'FECHA PAGO':'FechaPago','CodigoPedido':'CodigoPedido','MONTO':'ValorPagamento'})

  ###Formatar coluna de Pagamento

  consulta['ValorPagamento']=consulta['ValorPagamento'].astype(str)

  ###Formatar coluna de Data

  consulta['FechaPago']=consulta['FechaPago'].apply(lambda x: x[-2:]+'/'+x[2:4]+'/'+'20'+x[:2])

  ###Remover duplicados

  consulta.drop_duplicates(subset=consulta.columns,inplace=True)

  ###Adicionar índice

  consulta['ID']=consulta.index
  consulta['ID']=consulta['ID']+1
  consulta['ID']=consulta['ID'].astype(str).str.zfill(5)

  ##Exportando

  consulta.to_csv(csv_saida,index=False)

  ##Movendo Arquivos

  os.rename(dir_ingestion+nome_consulta,dir_processedfiles+nome_consulta)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2023-10-18


#**Pagos Natura Peru Padrão (com loop e sem validação de data - validando com débitos das carteiras de setembro)**

In [ ]:
for i in range(num_arquivo_ingestion[3]):
  #**Pagos Natura Peru Padrão (linkado com SQL)**

  import pandas as pd
  import numpy as np
  import os
  import time
  import datetime

  from google.colab import drive
  drive.mount('/content/drive')

  ##Definindo diretórios e nomes de arquivos

  #Diretório de saída do relatório
  carteira='Natura Peru'
  dir_pagos='/content/drive/MyDrive/Processamentos/Pagos/'
  dir_ingestion=dir_pagos+carteira+'/ingestion_natura-peru/'
  dir_processedfiles=dir_pagos+carteira+'/processedfiles/'
  dir_auxiliary=dir_pagos+carteira+'/auxiliary/'
  dir_saida=dir_pagos+'Payments_done_natura_peru/'
  diretorios=[dir_pagos,dir_ingestion,dir_processedfiles,dir_saida]
  for diretorio in diretorios:
    os.makedirs(diretorio,exist_ok=True)
  #Diretório dos arquivos tratados de hoje
  dir_tratados='/content/drive/MyDrive/Pronto pra upload/'

  #Procurando arquivo de Consulta na ingestion
  while len(os.listdir(dir_ingestion))==0:
    time.sleep(5)
  for arquivo in os.listdir(dir_ingestion):
    if arquivo.find('PAGOS')>=0:
      nome_consulta=arquivo

  '''#Pegando Data da Consulta
  data_consulta=datetime.date(int(nome_consulta[6:10]),int(nome_consulta[3:5]),int(nome_consulta[:2]))

  #Validando Data da Consulta
  today=datetime.date.today()
  if today.weekday()==0:
    dias=3
  else:
    dias=1'''
  '''if today-data_consulta>pd.Timedelta(days=dias):
    sys.exit()'''

  '''#Nome excel de saída
  excel_saida=dir_saida + str(data_consulta.strftime('%d-%m-%Y')) + '_pagos_natura-peru-exclusiva'+'.xlsx'
  print(data_consulta)'''

  ##Importação dos arquivos

  df_consulta=pd.read_excel(dir_ingestion+nome_consulta,dtype=object)

  import psycopg2
  conn = psycopg2.connect(database="sofi_homologacao",
                          host="db-teste-sofi.cs4lsziws4ie.us-east-2.rds.amazonaws.com",
                          user="usr_homologacao",
                          password="3cwvkLKcmSdZzg6",
                          port="5432")
  cursor = conn.cursor()
  cursor.execute('''select * from natura_pe_pedidos''')
  df_data=pd.DataFrame(cursor.fetchall())
  df_data.columns=['pedido']

  ##Criação de cópias

  consulta=df_consulta.copy()
  data=df_data.copy()

  ##Tratamento
  consulta['CodigoPedido']=consulta['CodigoPedido'].astype(str).str.strip()

  consulta=consulta.merge(data,left_on='CodigoPedido',right_on='pedido',how='inner')
  '''colunas=['data','FechaPago']
  for coluna in colunas:
    consulta[coluna]=pd.to_datetime(consulta[coluna])
  filtro=consulta['data']<=consulta['FechaPago']
  consulta=consulta[filtro]
  print(len(filtro)-filtro.sum(),'linhas excluídas')'''

  ###Renomear Colunas

  consulta=consulta.rename(columns={'FechaPago':'FechaPago','CodigoPedido':'CodigoPedido','ValorPagamento':'ValorPagamento'})

  ###Formatar coluna de Pagamento

  consulta['ValorPagamento']=consulta['ValorPagamento'].astype(str)

  ###Remover Duplicados

  consulta.drop_duplicates(subset=consulta.columns,inplace=True)

  ###Adicionar índice

  consulta['ID']=consulta.index
  consulta['ID']=consulta['ID']+1
  consulta['ID']=consulta['ID'].astype(str).str.zfill(5)

  ##Exportando

  consulta.to_excel(dir_saida + nome_consulta,index=False)  #excel_saida

  ##Movendo Arquivos

  os.rename(dir_ingestion+nome_consulta,dir_processedfiles+nome_consulta)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#**Relatorio de contas asignadas e retiradas Natura Mex Rep**

In [ ]:
import pandas as pd
import numpy as np
import os
import time
import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Copiar arquivo Rep para Relatórios

In [ ]:
import shutil
dir_relatorio='/content/drive/MyDrive/Processamentos/Relatorio/'
dir_rep='/content/drive/MyDrive/Processamentos/Tratamento/Natura MEX/processedfiles/'
for arquivo in os.listdir(dir_rep):
  if arquivo.find('Rep_CarteraApha_' + datetime.date.today().strftime('%d%m%Y'))>=0:
    nome_rep=arquivo
shutil.copyfile(dir_rep+nome_rep, dir_relatorio+nome_rep)

'/content/drive/MyDrive/Processamentos/Relatorio/Rep_CarteraApha_19102023_1247.csv'

##Definindo diretórios e nomes de arquivos

In [ ]:
#Diretório de saída do relatório
dir_relatorio='/content/drive/MyDrive/Processamentos/Relatorio/'
#Diretório dos arquivos tratados de hoje e o útlimo
dir_tratados='/content/drive/MyDrive/Pronto pra upload/'
today=datetime.date.today()
if today.weekday()==0:
  dias=3
else:
  dias=1
ultimo=today-datetime.timedelta(days=dias)
#ultimo=datetime.date(2023,10,11)
dir_hoje=dir_tratados+'/'.join((str(today.year),str(today.month).zfill(2),today.strftime('%d-%m-%Y')))+'/'
dir_ultimo=dir_tratados+'/'.join((str(ultimo.year),str(ultimo.month).zfill(2),ultimo.strftime('%d-%m-%Y')))+'/'
#Nome dos arquivos de hoje e ontem
nome_hoje='early_natura-mex-padrao_'+today.strftime('%d-%m-%Y')+'.csv'
nome_ultimo='early_natura-mex-padrao_'+ultimo.strftime('%d-%m-%Y')+'.csv'
#Nome excel de saída
excel_saida=dir_relatorio+'relatorio_natura-mex-padrao_'+today.strftime('%d-%m-%Y')+'.xlsx'

##Importação dos arquivos

In [ ]:
df_hoje=pd.read_csv(dir_hoje+nome_hoje,dtype=object)
df_ontem=pd.read_csv(dir_ultimo+nome_ultimo,dtype=object)

##Criação de cópias

In [ ]:
hoje=df_hoje.copy()
ontem=df_ontem.copy()

###Renomear colunas se existir

In [ ]:
if 'NÃºmeroPedido' in hoje.columns:
  hoje=hoje.rename(columns={'NÃºmeroPedido':'NúmeroPedido'})
if 'NÃºmeroPedido' in ontem.columns:
  ontem=ontem.rename(columns={'NÃºmeroPedido':'NúmeroPedido'})

##Mantendo colunas necessárias somente

In [ ]:
colunas=['Temporalidad Agencias','Saldo actualizado','NúmeroPedido']
hoje=hoje[colunas]
ontem=ontem[colunas]

##Filtrando Asignadas e Retiradas

In [ ]:
pedido_hoje=np.array(hoje['NúmeroPedido'])
pedido_ontem=np.array(ontem['NúmeroPedido'])
filtro_asignadas=np.logical_not(np.isin(pedido_hoje,pedido_ontem))
filtro_retiradas=np.logical_not(np.isin(pedido_ontem,pedido_hoje))
asignadas=hoje[filtro_asignadas]
retiradas=ontem[filtro_retiradas]

##Gerando Relatórios

In [ ]:
#Asignadas
asignadas.columns=['TEMPORALIDAD','SALDO ACTUALIZADO','CUENTAS']
asignadas['SALDO ACTUALIZADO']=asignadas['SALDO ACTUALIZADO'].astype(float)
contador=asignadas.groupby(by='TEMPORALIDAD',as_index=False)['CUENTAS'].count()
saldo=asignadas.groupby(by='TEMPORALIDAD',as_index=False)['SALDO ACTUALIZADO'].sum()
asignadas=contador.merge(saldo,on='TEMPORALIDAD')
asignadas['Tipo']='ASIGNADAS'

<ipython-input-192-97d62f0c389b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asignadas['SALDO ACTUALIZADO']=asignadas['SALDO ACTUALIZADO'].astype(float)


In [ ]:
#Retiradas
retiradas.columns=['TEMPORALIDAD','SALDO ACTUALIZADO','CUENTAS']
retiradas['SALDO ACTUALIZADO']=retiradas['SALDO ACTUALIZADO'].astype(float)
contador=retiradas.groupby(by='TEMPORALIDAD',as_index=False)['CUENTAS'].count()
saldo=retiradas.groupby(by='TEMPORALIDAD',as_index=False)['SALDO ACTUALIZADO'].sum()
retiradas=contador.merge(saldo,on='TEMPORALIDAD')
retiradas['Tipo']='RETIRADAS'

<ipython-input-193-5f05b1702034>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retiradas['SALDO ACTUALIZADO']=retiradas['SALDO ACTUALIZADO'].astype(float)


##Concatenando os dois relatórios

In [ ]:
relatorio=pd.concat([asignadas,retiradas])

##Exportando

In [ ]:
relatorio.to_excel(excel_saida,index=False)

#**Pagos Natura Mex Ambos (com loop)**

In [ ]:
for i in range(num_arquivo_ingestion[0]):
  #**Pagos Natura Mex Padrão**

  import pandas as pd
  import numpy as np
  import os
  import time
  import datetime

  from google.colab import drive
  drive.mount('/content/drive')

  ##Definindo diretórios e nomes de arquivos

  #Diretório de saída do relatório
  carteira='Natura MEX'
  dir_pagos='/content/drive/MyDrive/Processamentos/Pagos/'
  dir_ingestion=dir_pagos+carteira+'/ingestion_natura-mex/'
  dir_processedfiles=dir_pagos+carteira+'/processedfiles/'
  dir_saida=dir_pagos+'Payments_done_natura_mex/'
  #Diretório dos arquivos tratados de hoje
  dir_tratados='/content/drive/MyDrive/Pronto pra upload/'

  #Procurando arquivo de Consulta na ingestion
  while len(os.listdir(dir_ingestion))==0:
    time.sleep(5)
  for arquivo in os.listdir(dir_ingestion):
    if arquivo.find('Consulta')>=0:
      nome_consulta=arquivo
      print(nome_consulta)

  #Pegando Data da Consulta
  data_consulta=datetime.date(int(nome_consulta[6:10]),int(nome_consulta[3:5]),int(nome_consulta[:2]))

  #Validando Data da Consulta
  today=datetime.date.today()
  '''if today.weekday()==0:
    dias=3
  else:
    dias=1
  if today-data_consulta>pd.Timedelta(days=dias):
    sys.exit()'''

  #Gerando data da carteira
  if data_consulta.weekday()==4:
    dias=0
  elif data_consulta.weekday()==5:
    dias=1
  else:
    dias=-1
  data_carteira=data_consulta-pd.Timedelta(days=dias)
  #data_carteira=datetime.date(2023,10,11)

  #Nome dos arquivos de hoje e ontem
  nome_hoje='early_natura-mex-padrao_'+str(data_carteira.strftime('%d-%m-%Y'))+'.csv'
  dir_carteira=dir_tratados+str(data_carteira.year)+'/'+str(data_carteira.month).zfill(2)+'/'+str(data_carteira.strftime('%d-%m-%Y'))+'/'
  #Nome excel de saída
  excel_saida=dir_saida + str(data_consulta.strftime('%d-%m-%Y')) + '_pagos_natura-mex-padrao'+'.xlsx'
  print(data_carteira,data_consulta)

  ##Importação dos arquivos
  df_hoje=pd.read_csv(dir_carteira+nome_hoje,dtype=object)
  df_consulta=pd.read_csv(dir_ingestion+nome_consulta,dtype=object,sep='|') #

  ##Criação de cópias

  hoje=df_hoje.copy()
  consulta=df_consulta.copy()

  ##Mesclando e mantendo apenas colunas necessárias

  merged=consulta.merge(hoje,how='inner',left_on='PEDIDO',right_on='NúmeroPedido')
  merged=merged[['DOCUMENT_DATE','DT_PAGAMENTO','RECEIVER','DC_ENTIDADE_CONCILIACAO','CANAL_DE_DISTRIBUCION',\
                'PEDIDO','REGISTRO_PAGO','TITULO','REGISTRO_BAJA','TIPO_MOVIMIENTO','TIPO_BAJA','RECEIVER_BALANCE',\
                'PENALTY_INTEREST_BALANCE','LOSS_RECOVERY_BALANCE','TAX1','TAX2','TAX3','TAX4','TAX5','TAX6','DC_TIPO_RESULTADO',\
                'MOTIVO_REGISTRO_PAGAMENTO','PROCEDENCIA_REGISTRO_PAGAMENTO','DC_COMENTARIO']]

  ###Filtrando registros não pagos

  filtro=np.isin(merged['TIPO_BAJA'],['Estorno parcial titulo','Estorno total titulo','Título'])
  merged=merged[filtro]

  ###Renomear Colunas

  merged=merged.rename(columns={'DT_PAGAMENTO':'FechaPago','PEDIDO':'CodigoPedido','RECEIVER_BALANCE':'ValorPagamento'})

  ###Formatar coluna de Pagamento

  merged['ValorPagamento']=merged['ValorPagamento'].astype(str)

  ##Exportando

  merged.to_excel(excel_saida,index=False)


  #**Pagos Natura Mex Padrão**

  import pandas as pd
  import numpy as np
  import os
  import time
  import datetime

  from google.colab import drive
  drive.mount('/content/drive')

  ##Definindo diretórios e nomes de arquivos

  #Diretório de saída do relatório
  carteira='Natura MEX'
  dir_pagos='/content/drive/MyDrive/Processamentos/Pagos/'
  dir_ingestion=dir_pagos+carteira+'/ingestion_natura-mex/'
  dir_processedfiles=dir_pagos+carteira+'/processedfiles/'
  dir_saida=dir_pagos+'Payments_done_natura_mex/'
  #Diretório dos arquivos tratados de hoje
  dir_tratados='/content/drive/MyDrive/Pronto pra upload/'

  #Procurando arquivo de Consulta na ingestion
  while len(os.listdir(dir_ingestion))==0:
    time.sleep(5)
  for arquivo in os.listdir(dir_ingestion):
    if arquivo.find('Consulta')>=0:
      nome_consulta=arquivo
      print(nome_consulta)

  #Pegando Data da Consulta
  data_consulta=datetime.date(int(nome_consulta[6:10]),int(nome_consulta[3:5]),int(nome_consulta[:2]))

  #Validando Data da Consulta
  today=datetime.date.today()
  '''if today.weekday()==0:
    dias=3
  else:
    dias=1
  if today-data_consulta>pd.Timedelta(days=dias):
    sys.exit()'''

  #Gerando data da carteira
  if data_consulta.weekday()==4:
    dias=0
  elif data_consulta.weekday()==5:
    dias=1
  else:
    dias=-1
  data_carteira=data_consulta-pd.Timedelta(days=dias)
  #data_carteira=datetime.date(2023,10,11)
  #Nome dos arquivos de hoje e ontem
  nome_hoje='early_natura-mex-padrao_'+str(data_carteira.strftime('%d-%m-%Y'))+'.csv'
  dir_carteira=dir_tratados+str(data_carteira.year)+'/'+str(data_carteira.month).zfill(2)+'/'+str(data_carteira.strftime('%d-%m-%Y'))+'/'
  #Nome excel de saída
  excel_saida=dir_saida + str(data_consulta.strftime('%d-%m-%Y')) + '_pagos_natura-mex-padrao'+'.xlsx'
  print(data_carteira,data_consulta)

  ##Importação dos arquivos

  df_hoje=pd.read_csv(dir_carteira+nome_hoje,dtype=object)
  df_consulta=pd.read_csv(dir_ingestion+nome_consulta,dtype=object,sep='|') #

  ##Criação de cópias

  hoje=df_hoje.copy()
  consulta=df_consulta.copy()

  ##Mesclando e mantendo apenas colunas necessárias

  merged=consulta.merge(hoje,how='inner',left_on='PEDIDO',right_on='NúmeroPedido')
  merged=merged[['DOCUMENT_DATE','DT_PAGAMENTO','RECEIVER','DC_ENTIDADE_CONCILIACAO','CANAL_DE_DISTRIBUCION',\
                'PEDIDO','REGISTRO_PAGO','TITULO','REGISTRO_BAJA','TIPO_MOVIMIENTO','TIPO_BAJA','RECEIVER_BALANCE',\
                'PENALTY_INTEREST_BALANCE','LOSS_RECOVERY_BALANCE','TAX1','TAX2','TAX3','TAX4','TAX5','TAX6','DC_TIPO_RESULTADO',\
                'MOTIVO_REGISTRO_PAGAMENTO','PROCEDENCIA_REGISTRO_PAGAMENTO','DC_COMENTARIO']]

  ###Filtrando registros não pagos

  filtro=np.isin(merged['TIPO_BAJA'],['Estorno parcial titulo','Estorno total titulo','Título'])
  merged=merged[filtro]

  ###Renomear Colunas

  merged=merged.rename(columns={'DT_PAGAMENTO':'FechaPago','PEDIDO':'CodigoPedido','RECEIVER_BALANCE':'ValorPagamento'})

  ###Formatar coluna de Pagamento

  merged['ValorPagamento']=merged['ValorPagamento'].astype(str)

  ###Adicionar índice
  merged['ID']=merged.index
  merged['ID']=merged['ID']+1
  merged['ID']=merged['ID'].astype(str).str.zfill(5)

  ##Exportando

  merged.to_excel(excel_saida,index=False)







  #**Pagos Natura Mex Tardia**
  indice_0_tardia=len(merged)

  import pandas as pd
  import numpy as np
  import os
  import time
  import datetime

  from google.colab import drive
  drive.mount('/content/drive')

  ##Definindo diretórios e nomes de arquivos

  #Diretório de Descontos
  dir_descontos='/content/drive/MyDrive/Processamentos/Pagos/_Descontos/'

  #Data da consulta
  data_consulta=datetime.date(int(nome_consulta[6:10]),int(nome_consulta[3:5]),int(nome_consulta[:2]))

  #Nome excel de saída
  excel_saida=dir_saida+str(data_consulta.strftime('%d-%m-%Y'))+'_pagos_natura-mex-tardia' +'.xlsx'
  print(data_consulta)
  #Nome saída do arquivo de Descontos
  descontos_saida=dir_descontos+'descontos_natura-mex-tardia_'+str(data_consulta.strftime('%d-%m-%Y')) +'.xlsx'

  ##Importação dos arquivos

  #Procurar última carteira Late Stage
  carteira='natura-mex-tardia'

  pasta=''
  achar=False
  for i in range(100):
    data=today-pd.Timedelta(days=i)
    diretorio= '/content/drive/MyDrive/Pronto pra upload/'+str(data.year)+'/'+str(data.month).zfill(2)+'/'+data.strftime('%d-%m-%Y')+'/'
    try:
      for arquivo in os.listdir(diretorio):
        if arquivo.find(carteira)>=0:
          achar=True
          tardia=diretorio+arquivo
          nome_tardia=arquivo
          break
    except:
      continue
    if achar:
      break
  #Carregar última carteira Late Stage
  df_hoje=pd.read_csv(tardia,dtype=object)

  ##Criação de cópias

  hoje=df_hoje.copy()
  consulta=df_consulta.copy()

  ##Mesclando e mantendo apenas colunas necessárias

  merged=consulta.merge(hoje,how='inner',left_on='PEDIDO',right_on='NúmeroPedido')
  merged=merged[['DOCUMENT_DATE','DT_PAGAMENTO','RECEIVER','DC_ENTIDADE_CONCILIACAO','CANAL_DE_DISTRIBUCION',\
                'PEDIDO','REGISTRO_PAGO','TITULO','REGISTRO_BAJA','TIPO_MOVIMIENTO','TIPO_BAJA','RECEIVER_BALANCE',\
                'PENALTY_INTEREST_BALANCE','LOSS_RECOVERY_BALANCE','TAX1','TAX2','TAX3','TAX4','TAX5','TAX6','DC_TIPO_RESULTADO',\
                'MOTIVO_REGISTRO_PAGAMENTO','PROCEDENCIA_REGISTRO_PAGAMENTO','DC_COMENTARIO','Dias de morosidad','CódigoPersona',\
                'Saldo Total','Saldo actualizado','Saldo Principal']]

  ###Filtrando registros não pagos

  filtro=np.isin(merged['TIPO_BAJA'],['Estorno parcial titulo','Estorno total titulo','Título'])
  merged=merged[filtro]

  ##Gerar Tabela de Descontos

  colunas=['CódigoPersona','PEDIDO','Saldo Total','Dias de morosidad','RECEIVER_BALANCE','DT_PAGAMENTO','Saldo actualizado','Saldo Principal']
  descontos=merged[colunas]

  ###Filtrar Saldo Total <200

  filtro=descontos['Saldo Total'].astype(float)>=200
  descontos=descontos[filtro]

  ###Concatenar Débitos da carteira com Código Persona (Consultora) que estão na Lista de Descontos

  #Filtrar débitos com mesmo CódigoPersona e NúmeroPedido diferente dos débitos que estão na Lista de Descontos
  filtro1=hoje['CódigoPersona'].isin(descontos['CódigoPersona'])
  filtro2=np.logical_not(hoje['NúmeroPedido'].isin(descontos['PEDIDO']))
  filtro=filtro1*filtro2
  consulta=hoje[filtro]
  #Manter Colunas Necessárias Somente
  colunas=['CódigoPersona','NúmeroPedido','Saldo Total','Dias de morosidad','Saldo actualizado','Saldo Principal']
  consulta=consulta[colunas]
  consulta=consulta.rename(columns={'NúmeroPedido':'PEDIDO'})
  descontos=pd.concat([descontos,consulta],join='outer')

  ###Adicionando Colunas

  #Adicionando: Descuento ofrecido de Multa
  descontos['Descuento ofrecido de Multa']='100%'

  #Adicionando: Descuento ofrecido de Capital
  filtro=pd.to_numeric(descontos['Dias de morosidad'])<=1825
  descontos.loc[filtro,'Descuento ofrecido de Capital']=0.4
  filtro=pd.to_numeric(descontos['Dias de morosidad'])<=1080
  descontos.loc[filtro,'Descuento ofrecido de Capital']=0.35
  filtro=pd.to_numeric(descontos['Dias de morosidad'])<=720
  descontos.loc[filtro,'Descuento ofrecido de Capital']=.25
  filtro=pd.to_numeric(descontos['Dias de morosidad'])<=360
  descontos.loc[filtro,'Descuento ofrecido de Capital']=0.2

  #Adicionando: Total a pagar por la Consultora
  descontos['Total a pagar por la Consultora']=filtro=descontos['Saldo Total'].astype(float)*(1-descontos['Descuento ofrecido de Capital'])

  #Adicionando: Valor do Desconto/Multa
  colunas=['Saldo actualizado','Saldo Principal']
  for coluna in colunas:
    descontos[coluna]=descontos[coluna].astype(float)
  descontos['Valor do Desconto/Multa']=descontos['Saldo actualizado']-descontos['Saldo Principal']

  #Adicionando: Valor do Desconto/Principal
  descontos['Valor do Desconto/Principal']=descontos['Descuento ofrecido de Capital']*descontos['Saldo Principal']

  ###Excluir colunas

  #São colunas usadas para calcular
  colunas=['Saldo actualizado','Saldo Principal']
  descontos.drop(colunas, axis=1, inplace=True)

  ###Reordenar

  ordem=['CódigoPersona','PEDIDO','Saldo Total','Descuento ofrecido de Multa','Dias de morosidad','Descuento ofrecido de Capital','Total a pagar por la Consultora','RECEIVER_BALANCE','DT_PAGAMENTO','Valor do Desconto/Multa','Valor do Desconto/Principal']
  descontos=descontos[ordem]

  ###Renomear

  colunas=['Consultora','Pedido','Saldo pendiente','Descuento ofrecido de Multa','Aging','Descuento ofrecido de Capital','Total a pagar por la Consultora','PAGADO','FECHA DE PAGO','Valor do Desconto/Multa','Valor do Desconto/Principal']
  descontos.columns=colunas

  ##Renomear colunas do arquivo de pagamento

  merged=merged.rename(columns={'DT_PAGAMENTO':'FechaPago','PEDIDO':'CodigoPedido','RECEIVER_BALANCE':'ValorPagamento'})

  ###Formatar colunas de Valores de Descontos

  colunas=['Saldo pendiente','Descuento ofrecido de Capital','Total a pagar por la Consultora','PAGADO','Valor do Desconto/Multa','Valor do Desconto/Principal']
  for coluna in colunas:
    descontos[coluna]=descontos[coluna].astype(float).round(2).map('{:.2f}'.format).astype(str)

  ###Formatar coluna de Pagamento

  merged['ValorPagamento']=merged['ValorPagamento'].astype(str)

  ###Adicionar índice
  merged['ID']=merged.index
  merged['ID']=merged['ID']+indice_0_tardia+1
  merged['ID']=merged['ID'].astype(str).str.zfill(5)

  ##Exportando

  merged.to_excel(excel_saida,index=False)
  descontos.to_excel(descontos_saida,index=False)

  ##Movendo Arquivos

  os.rename(dir_ingestion+nome_consulta,dir_processedfiles+nome_consulta)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
18-10-2023_Consulta_Q1390_P5115922_20231019114619230.csv
2023-10-19 2023-10-18
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
18-10-2023_Consulta_Q1390_P5115922_20231019114619230.csv
2023-10-19 2023-10-18
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2023-10-18


#**Datos Telefonicos**

In [ ]:
import pandas as pd
import os
import datetime

# Natura PE Exclusiva

today=datetime.date.today()
dir='/'.join(['/content/drive/MyDrive/Pronto pra upload',str(today.year),str(today.month).zfill(2),today.strftime('%d-%m-%Y')])+'/''
nome_datos='/content/drive/MyDrive/Processamentos/Tratamento/auxiliary/Datos telefónicos - HISOFI 16.10.xlsx'
for arquivo in os.listdir(dir):
    if arquivo.find('natura-peru-exclusiva')>=0:
        nome_carteira=dir+arquivo
carteira=pd.read_csv(nome_carteira,dtype=str)
print(len(carteira))

datos=pd.read_excel(nome_datos,sheet_name='NATURA',dtype=str)

datos=datos[['DNI','CELULAR 1']]
datos=datos.drop_duplicates('DNI')

import psycopg2
conn = psycopg2.connect(database="sofi_homologacao",
                        host="db-teste-sofi.cs4lsziws4ie.us-east-2.rds.amazonaws.com",
                        user="usr_homologacao",
                        password="3cwvkLKcmSdZzg6",
                        port="5432")
cursor = conn.cursor()
sql='''SELECT distinct offers."document"  FROM "agreements"
INNER JOIN "agreement_items" ON "agreement_items"."agreement_id" = "agreements"."id"
INNER JOIN "offers" ON "offers"."id" = "agreement_items"."offer_id"
where wallet_id in (21,164)
and agreements.status is not null
union all
select distinct "number"
from user_documents
where number in (select document from offers where wallet_id in (21,164))'''
cursor.execute(sql)
match_acordo=pd.DataFrame(cursor.fetchall(),columns=['document'])

#Tirar documentos com match ou acordo
filtro=~datos['DNI'].isin(match_acordo['document'])
datos=datos[filtro]

match_acordo['document']=match_acordo['document'].astype(str)

merged=carteira.merge(datos,on='DNI',how='left')

filtro=~merged['CELULAR 1'].isnull()
merged.loc[filtro,'Numero Celular']=merged.loc[filtro,'CELULAR 1']

filtro.sum()

print(len(merged))

merged.to_csv(nome_carteira,index=False)

# Avon PE

for arquivo in os.listdir(dir):
    if arquivo.find('avon-pe-padrao')>=0:
        nome_carteira=dir+arquivo
carteira=pd.read_csv(nome_carteira,dtype=str)
print(len(carteira))

datos=pd.read_excel(nome_datos,sheet_name='AVON',dtype=str)

datos=datos[['DNI','CELULAR 1']]
datos=datos.drop_duplicates('DNI')

#Tirar documentos com match ou acordo
filtro=~datos['DNI'].isin(match_acordo['document'])
datos=datos[filtro]

carteira['DNI']=carteira['TERRITORIO'].str[3:]

merged=carteira.merge(datos,on='DNI',how='left')

len(merged)

merged.columns

filtro=~merged['CELULAR 1'].isnull()
merged.loc[filtro,'TELEFONO 1']=merged.loc[filtro,'CELULAR 1']

filtro.sum()

print(len(merged))

merged.to_csv(nome_carteira,index=False)